In [2]:
import gzip
import json
import re
import os
import sys
import numpy as np
import pandas as pd

In [10]:
#https://github.com/MengtingWan/goodreads/blob/master/samples.ipynb
def load_data_books(file_name, head = None):
    count = 0
    data = []
    with gzip.open(file_name) as fin:
        for l in fin:
            d = json.loads(l)
            authors = d['authors']
            aut_ids = []
            for author in authors:
                aut_ids.append(author['author_id'])
            item = {'book_id': d['book_id'], 
                    'title':d['title'], 
                    'author_ids':aut_ids, 
                    'average_rating':d['average_rating'],
                    'ratings_count':d['ratings_count']}
            count += 1
            data.append(item)
    
            # break if reaches head
            if (head is not None) and (count > head):
                break
            
    return data

In [8]:
#https://github.com/MengtingWan/goodreads/blob/master/samples.ipynb
def load_data_reviews(file_name, head = None):
    count = 0
    data = []
    with gzip.open(file_name) as fin:
        for l in fin:
            d = json.loads(l)
            item = {'book_id': d['book_id'], 'rating':d['rating'], 'review_text':d['review_text']}
            count += 1
            data.append(item)
    
            # break if reaches head
            if (head is not None) and (count > head):
                break
            
    return data

In [5]:
#https://github.com/MengtingWan/goodreads/blob/master/samples.ipynb
def load_data_authors(file_name, head = None):
    count = 0
    data = []
    with gzip.open(file_name) as fin:
        for l in fin:
            d = json.loads(l)
            item = {'id': d['author_id'], 'name':d['name']}
            count += 1
            data.append(item)
    
            # break if reaches head
            if (head is not None) and (count > head):
                break
            
    return data

In [11]:
DIR = './'

BOOKS = load_data_books(os.path.join(DIR, 'goodreads_books.json.gz'))

In [12]:
print(BOOKS[5])

{'book_id': '287141', 'title': 'The Aeneid for Boys and Girls', 'author_ids': ['3041852'], 'average_rating': '4.13', 'ratings_count': '46'}


In [13]:
print(BOOKS[1]['book_id'],BOOKS[1]['title'], BOOKS[1]['average_rating'])
print(len(BOOKS))

1333909 Good Harbor 3.23
2360655


In [14]:
REVIEWS = load_data_reviews(os.path.join(DIR, 'goodreads_reviews_dedup.json.gz'))

In [15]:
print(REVIEWS[2])
print(len(REVIEWS))

{'book_id': '6392944', 'rating': 3, 'review_text': "I haven't read a fun mystery book in a while and not sure I've ever read Poirot. Was looking for a fun read set in France while I was on holiday there and this didn't disappoint! Fast paced and good mystery. \n One that struck me was how similar Poirot is to Sherlock. They are both detectives, have a ex-military sidekick who is telling the story, and solve mysteries using their superior wit. Poirot seems like a French Sherlock. I'm curious if he was inspired by Sherlock."}
15739967


In [16]:
AUTHORS = load_data_authors(os.path.join(DIR, 'goodreads_book_authors.json.gz'))

In [17]:
print(AUTHORS[1])

{'id': '626222', 'name': 'Anita Diamant'}


## Create data

In [18]:
stories = []
with open("short_stories_under_2k.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        stories.append(json.loads(line))

In [20]:
def find_id_by_title_2(title_to_find, author):
    skip = False
    ids = []
    total_ratings = 0
    count = 0
    author_ids = []
    for item in BOOKS:
        if title_to_find in item['title']:
            if title_to_find == item['title']:
                names = []
                for aut_id in item['author_ids']:
                    name = find_name_by_id(aut_id)
                    if name:
                        names.append(name)
                    
                if author not in names:
                    print('Check this makes sense for', title_to_find, ':', author, names)
                    response = input('Does this make sense y/n? ')
                    if response == 'n':
                        continue
            elif (skip == False):
                print('Compare books:', title_to_find,' by ',author, ' , ',item['title'])
                response = input('Are these the same? y/n ')
                if response == 'n':
                    continue
                elif response == 's':
                    skip = True
                    continue

            ids.append(item['book_id'])
            try:
                rating_av = float(item['average_rating'])
                num = int(item['ratings_count'])
                total_ratings += rating_av * num
                count += num
            except (ValueError, TypeError, KeyError):
                pass
            
            for aut in item['author_ids']:
                if aut not in author_ids:
                    author_ids.append(aut)
    if count == 0:
        average_rating = 0
    else:
        average_rating = total_ratings / count
    return ids, average_rating, author_ids, count


def find_info_by_title(title_to_find,author):
    ids = []
    total_ratings = 0
    count = 0
    author_ids = []
    for item in BOOKS:
        if title_to_find == item['title']:
            names = []
            for aut_id in item['author_ids']:
                name = find_name_by_id(aut_id)
                if name:
                    names.append(name)
                
            if author not in names:
                continue


            ids.append(item['book_id'])
            try:
                rating_av = float(item['average_rating'])
                num = int(item['ratings_count'])
                total_ratings += rating_av * num
                count += num
            except (ValueError, TypeError, KeyError):
                pass
            
            for aut in item['author_ids']:
                if aut not in author_ids:
                    author_ids.append(aut)
                    
    if count == 0:
        average_rating = 0
    else:
        average_rating = total_ratings / count
    return ids, average_rating, author_ids, count

def find_reviews_by_id(ids):
    this_book_reviews = []
    for id in ids:
        for item in REVIEWS:
            if item['book_id'] == id:
                review = {'rating': item['rating'], 'text': item['review_text']}
                this_book_reviews.append(review)
    return this_book_reviews

def find_name_by_id(id):
    for author in AUTHORS:
        if author['id'] == id:
            return author['name']
    return None




In [21]:

# Main processing loop
unfound = []
data = []

for story in stories:
    # Find book id
    ids, av_rating, author_ids, num_ratings = find_info_by_title(story['title'],story['author'])
    if ids == []:
        print(story['title'], 'by', story['author'], ' not found in Goodreads')
        unfound.append(story)
        continue

    # Get the reviews
    book_reviews = find_reviews_by_id(ids)

    # Combine into final record
    book = {
        'id': ids,
        'title': story['title'],
        'author': story['author'],
        'story': story['text'],
        'av_rating': av_rating,
        'num_ratings': num_ratings,
        'reviews': book_reviews
    }
    data.append(book)

with open("all_data_together_auto_with_count.jsonl", "w", encoding="utf-8") as f:
        for book in data:
            f.write(json.dumps(book, ensure_ascii=False) + "\n")

The Child’s Story by Charles Dickens  not found in Goodreads
The Bookshop by Aldous Huxley  not found in Goodreads
Dead Woman’s Secret by Guy de Maupassant  not found in Goodreads
The Little Red Christmas Tree by Aleksandr I. Kuprin  not found in Goodreads
The Land of Counterpane by Robert Louis Stevenson  not found in Goodreads
Clancy of the Overflow by Banjo Paterson  not found in Goodreads
Old Pardon, the Son of Reprieve by Banjo Paterson  not found in Goodreads
Black Swans by Banjo Paterson  not found in Goodreads
An Idyll of Dandaloo by Banjo Paterson  not found in Goodreads
The Dog by Banjo Paterson  not found in Goodreads
The Virgin by Rudyard Kipling  not found in Goodreads
A Diagnosis of Death by Ambrose Bierce  not found in Goodreads
Praise the Generous Gods by William Ernest Henley  not found in Goodreads
The Eagle and the Fox by Aesop  not found in Goodreads
The North Wind and the Sun by Aesop  not found in Goodreads
The Crab and His Mother by Aesop  not found in Goodreads


In [22]:
print(len(data))

57


## Manually check for ones not found

In [25]:
def find_id_by_title_3(title_to_find,author):
    skip = False
    ids = []
    total_ratings = 0
    count = 0
    author_ids = []
    for item in BOOKS:
        if title_to_find == item['title']:
            names = []
            for aut_id in item['author_ids']:
                name = find_name_by_id(aut_id)
                if name:
                    names.append(name)
                
            if author not in names:
                if skip == True:
                    continue
                else:
                    print('Check this makes sense for', title_to_find, ':', author, names)
                    response = input('Does this make sense y/n? ')
                    if response == 'n':
                        continue
                    elif response == 's':
                        skip = True
                        continue

            ids.append(item['book_id'])
            try:
                rating_av = float(item['average_rating'])
                num = int(item['ratings_count'])
                total_ratings += rating_av * num
                count += num
            except (ValueError, TypeError, KeyError):
                pass
            
            for aut in item['author_ids']:
                if aut not in author_ids:
                    author_ids.append(aut)
                    
    if count == 0:
        average_rating = 0
    else:
        average_rating = total_ratings / count
    
    return ids, average_rating, author_ids, count

In [26]:
still_unfound = []
data_2 = []

for story in unfound:
    # Find book id
    ids, av_rating, author_ids, num_ratings = find_id_by_title_3(story['title'],story['author'])
    if ids == []:
        print(story['title'], 'by', story['author'], ' not found in Goodreads')
        still_unfound.append(story)
        continue

    # Get the REVIEWS
    book_reviews = find_reviews_by_id(ids)

    # Combine into final record
    book = {
        'id': ids,
        'title': story['title'],
        'author': story['author'],
        'story': story['text'],
        'av_rating': av_rating,
        'num_ratings': num_ratings,
        'reviews': book_reviews
    }
    data_2.append(book)

The Child’s Story by Charles Dickens  not found in Goodreads
Check this makes sense for The Bookshop : Aldous Huxley ['Penelope Fitzgerald']


Does this make sense y/n?  s


The Bookshop by Aldous Huxley  not found in Goodreads
Dead Woman’s Secret by Guy de Maupassant  not found in Goodreads
The Little Red Christmas Tree by Aleksandr I. Kuprin  not found in Goodreads
The Land of Counterpane by Robert Louis Stevenson  not found in Goodreads
Clancy of the Overflow by Banjo Paterson  not found in Goodreads
Old Pardon, the Son of Reprieve by Banjo Paterson  not found in Goodreads
Check this makes sense for Black Swans : Banjo Paterson ['Eve Babitz']


Does this make sense y/n?  s


Black Swans by Banjo Paterson  not found in Goodreads
An Idyll of Dandaloo by Banjo Paterson  not found in Goodreads
Check this makes sense for The Dog : Banjo Paterson ['Amy Cross']


...

Does this make sense y/n?  i


Check this makes sense for The Terrible Old Man : H. P. Lovecraft ['H.P. Lovecraft']


Does this make sense y/n?  i


Check this makes sense for The Terrible Old Man : H. P. Lovecraft ['H.P. Lovecraft']


Does this make sense y/n?  i


In [27]:
print(len(data_2))

25


In [28]:
data.extend(data_2)
print(len(data))

82


In [29]:
with open("all_data_together_auto_and_some_added_with_count.jsonl", "w", encoding="utf-8") as f:
        for book in data:
            f.write(json.dumps(book, ensure_ascii=False) + "\n")

## Really Manually

In [30]:
import copy
data_3 = copy.deepcopy(data)

In [31]:
for book in still_unfound:
    print(f"{book['title']}, by {book['author']}")

The Child’s Story, by Charles Dickens
The Bookshop, by Aldous Huxley
Dead Woman’s Secret, by Guy de Maupassant
The Little Red Christmas Tree, by Aleksandr I. Kuprin
The Land of Counterpane, by Robert Louis Stevenson
Clancy of the Overflow, by Banjo Paterson
Old Pardon, the Son of Reprieve, by Banjo Paterson
Black Swans, by Banjo Paterson
An Idyll of Dandaloo, by Banjo Paterson
The Dog, by Banjo Paterson
The Virgin, by Rudyard Kipling
A Diagnosis of Death, by Ambrose Bierce
Praise the Generous Gods, by William Ernest Henley
The Eagle and the Fox, by Aesop
The North Wind and the Sun, by Aesop
The Crab and His Mother, by Aesop
The Leopard Man’s Story, by Jack London
The Miller, His Son, and Their Ass, by Aesop
Mr. and Mrs. Vinegar, by Flora Annie Steel
Demir-kayá, by Aleksandr I. Kuprin
The Farmer and the Fox, by Aesop
Come-By-Chance, by Banjo Paterson
The Laidly Worm, by Flora Annie Steel
The Boar-Pig, by Saki
The Recovery, by Kate Chopin
A Love by The Sea, by William Ernest Henley
The W

In [32]:
def find_id_by_name(name):
    for author in AUTHORS:
        if author['name'] == name:
            return author['id']

def find_books_by_author(author):
    titles = []
    id = find_id_by_name(author)
    for book in BOOKS:
        if(id in book['author_ids']):
            titles.append([book['title'],book['book_id']])
    return titles

### The Child's Story

In [35]:
chls_dic_bks = find_books_by_author('Charles Dickens')
#print(chls_dic_bks)

In [36]:
matches = [s for s in chls_dic_bks if 'Child' in s[0]]
print(matches)

[["The Child's Story", '922878'], ["Children's Stories from Dickens", '329943'], ["A Child's Dream of a Star", '10975557'], ["A Child's History of England", '1775352'], ["Best in Children's Books, Volume 4", '17157765'], ['Holiday Romance and Other Writings for Children', '5661'], ['The Life of Our Lord: Written for His Children During the Years 1846 to 1849', '5342'], ['The Life of Our Lord: Written for His Children During the Years 1846 to 1849', '25108031'], ["A Child's Dream of a Star", '11271813'], ["A Child's History Of England", '11459671'], ["A Child's History of England", '26223129'], ["The Children's Dickens - Great Expectations", '9827865'], ["A Child's History of England", '22093452'], ['Classics to Read Aloud to Your Children: Selections from Shakespeare, Twain, Dickens, O.Henry, London, Longfellow, Irving Aesop, Homer, Cervantes, Hawthorne, and More', '110463'], ["The Viking Treasury of Children's Stories", '4917036'], ["Charles Dickens' Children Stories", '12490289'], ["

Ids: 922878, 23844713

In [37]:
new_ids = ['922878', '23844713']

total_ratings = 0
count = 0
for id in new_ids:
    for item in BOOKS:
        if id == item['book_id']:
            try:
                rating_av = float(item['average_rating'])
                num = int(item['ratings_count'])
                total_ratings += rating_av * num
                count += num
            except (ValueError, TypeError, KeyError):
                pass
                    
if count == 0:
    average_rating = 0
else:
    average_rating = total_ratings / count

story = still_unfound[0]
print(story['title'])
book_reviews = find_reviews_by_id(new_ids)

# Combine into final record
book = {
    'id': new_ids,
    'title': story['title'],
    'author': story['author'],
    'story': story['text'],
    'av_rating': average_rating,
    'num_ratings':count,
    'reviews': book_reviews
}

data_3.append(book)

The Child’s Story


In [38]:
print(len(data_3))

83


#### The BOOKShop

In [139]:
still_unfound[1]['title']

'The Bookshop'

not there

#### A dead woman's secret

In [142]:
still_unfound[2]['title']

'Dead Woman’s Secret'

In [39]:
story = still_unfound[2]
for item in BOOKS:
    if item['title'] == "A Dead Woman's Secret":
        new_ids = [item['book_id']]
        average_rating = item['average_rating']
        num_ratings = item['ratings_count']
        
book_reviews = find_reviews_by_id(new_ids)
book = {
    'id': new_ids,
    'title': "A Dead Woman's Secret",
    'author': story['author'],
    'story': story['text'],
    'av_rating': average_rating,
    'num_ratings':num_ratings,
    'reviews': book_reviews
}

data_3.append(book)

In [40]:
print(book)

{'id': ['35993742'], 'title': "A Dead Woman's Secret", 'author': 'Guy de Maupassant', 'story': 'The woman had died without pain, quietly, as a woman should whose life had been blameless. Now she was resting in her bed, lying on her back, her eyes closed, her features calm, her long white hair carefully arranged as though she had done it up ten minutes before dying. The whole pale countenance of the dead woman was so collected, so calm, so resigned that one could feel what a sweet soul had lived in that body, what a quiet existence this old soul had led, how easy and pure the death of this parent had been.\n\nKneeling beside the bed, her son, a magistrate with inflexible principles, and her daughter, Marguerite, known as Sister Eulalie, were weeping as though their hearts would break. She had, from childhood up, armed them with a strict moral code, teaching them religion, without weakness, and duty, without compromise. He, the man, had become a judge and handled the law as a weapon with

#### The Little Red Christmas Tree

In [166]:
still_unfound[3]['title']

'The Little Red Christmas Tree'

not there

#### The land of counterpane

In [172]:
still_unfound[4]['title']

'The Land of Counterpane'

In [42]:
rls =find_books_by_author('Robert Louis Stevenson')
matches = [s for s in rls if 'Land' in s[0]]
print(matches)

[['The Little Land', '3047241'], ['The Land of Nod', '29633854']]


still not there

#### Clancy of the Overflow

In [174]:
still_unfound[5]['title']

'Clancy of the Overflow'

In [43]:
find_books_by_author('A.B. Paterson')


[['The Merino Sheep', '33391789'],
 ['Complete Poems', '745346'],
 ['Twas the Night Before Christmas and Other Christmas Stories', '8468620'],
 ["We're All Australians Now", '31847636'],
 ['Poems of Banjo Paterson', '20995192'],
 ["We're All Australians Now", '24358240'],
 ['The Cat', '23633435'],
 ['Waltzing Matilda', '111281'],
 ['The Best Of Banjo Patterson: An Illustrated Collection', '3172903'],
 ['The Man from Snowy River', '1841075'],
 ['Saltbush Bill, J. P', '12255399'],
 ['The Man From Snowy River', '6072732'],
 ["The Banjo's Best-Loved Poems", '11480077'],
 ["Works of 'Banjo' Paterson", '2746647'],
 ['The Man from Snowy River and Other Verses', '2725644'],
 ["Mulga Bill's Bicycle", '1651879'],
 ['The Man from Snowy River', '7902381'],
 ['An Outback Marriage', '6512349'],
 ['Clancy Of The Overflow', '397268'],
 ['Waltzing Matilda', '16074139']]

In [44]:
new_ids = ['397268']

total_ratings = 0
count = 0
for id in new_ids:
    for item in BOOKS:
        if id == item['book_id']:
            try:
                rating_av = float(item['average_rating'])
                num = int(item['ratings_count'])
                total_ratings += rating_av * num
                count += num
            except (ValueError, TypeError, KeyError):
                pass
                    
if count == 0:
    average_rating = 0
else:
    average_rating = total_ratings / count

story = still_unfound[5]
print(story['title'])
book_reviews = find_reviews_by_id(new_ids)

# Combine into final record
book = {
    'id': new_ids,
    'title': story['title'],
    'author': story['author'],
    'story': story['text'],
    'av_rating': average_rating,
    'num_ratings':count,
    'reviews': book_reviews
}

#print(book)
data_3.append(book)

Clancy of the Overflow


In [45]:
print(len(data_3))

85


#### Old Pardon, the son of reprieve

In [182]:
still_unfound[6]['title']

'Old Pardon, the Son of Reprieve'

not there

#### Black swans

In [184]:
still_unfound[7]['title']

'Black Swans'

not there

#### Idyll of Dandaloo

In [186]:
still_unfound[8]['title']

'An Idyll of Dandaloo'

#### The Dog

In [189]:
still_unfound[9]['title']

'The Dog'

#### The Virgin

In [192]:
still_unfound[10]['title']

'The Virgin'

#### A Diagnosis of Death

In [195]:
still_unfound[11]['title']

'A Diagnosis of Death'

In [199]:
# it exists but just can't seem to find it by computer

In [47]:
story = still_unfound[11]
for item in BOOKS:
    if item['title'] == "A Diagnosis of Death":
        print(item['book_id'])
        #new_ids = [item['book_id']]
        #average_rating = item['average_rating']
book = {
    'id': [],
    'title': story['title'],
    'author': story['author'],
    'story': story['text'],
    'av_rating': 3.21,
    'num_ratings':119,
    'reviews': [{'rating':4, 'text':"A very atmospheric and creepy story. Hawver meets a Dr who can foresee when death is coming. What about his own life? This gothic tale will run shivers down your spine. Really recommended!"},
               {'rating':3,'text':"Good, but not great, and certainly with a few stylistic hiccups that I did not particularly care for; nonetheless, however- in concept especially- this is a perfectly pleasant little read. It just so happens to not be anything worth writing home about, in my opinion." },
               {'rating':2,'text':" This is such a short short-story that it doesn't have enough time to scare us!"}
               ]
}

#print(book)
data_3.append(book)

#### Praise the generous gods

In [204]:
still_unfound[12]

{'title': 'Praise the Generous Gods',
 'author': 'William Ernest Henley',
 'text': 'Praise the generous gods for giving\n\nIn a world of wrath and strife\n\nWith a little time for living,\n\nUnto all the joy of life.\n\nAt whatever source we drink it,\n\nArt or love or faith or wine,\n\nIn whatever terms we think it,\n\nIt is common and divine.\n\nPraise the high gods, for in giving\n\nThis to man, and this alone,\n\nThey have made his chance of living\n\nShine the equal of their own.',
 'word_count': 75,
 'url': 'https://www.libraryofshortstories.com/onlinereader/praise-the-generous-gods'}

#### The eagle and the fox

In [205]:
still_unfound[13]['title']

'The Eagle and the Fox'

In [48]:
ae = find_books_by_author('Aesop')
matches = [s for s in ae if 'fox' in s[0].lower()]
print(matches)

[['Fox Tails: Four Fables from Aesop', '12710065'], ['The Fox and the Goat', '36437604'], ['The Fox And The Lion', '33131735'], ['The Fox And The Grapes', '33131659'], ['The Bear And The Fox', '36437784'], ['The Eagle And The Fox', '36434944'], ['The Lion And The Fox', '36448128'], ['What The Fox Learnt: Four Fables from Aesop', '12574862'], ['The Dog The Cock And The Fox', '32805954']]


In [50]:
for item in BOOKS:
    if item['book_id'] =='36434944':
        names = []
        for aut_id in item['author_ids']:
            name = find_name_by_id(aut_id)
            names.append(name)
print(names)

['Aesop']


In [51]:
new_ids = ['36434944']
total_ratings = 0
count = 0
for id in new_ids:
    for item in BOOKS:
        if id == item['book_id']:
            try:
                rating_av = float(item['average_rating'])
                num = int(item['ratings_count'])
                total_ratings += rating_av * num
                count += num
            except (ValueError, TypeError, KeyError):
                pass
                    
if count == 0:
    average_rating = 0
else:
    average_rating = total_ratings / count

story = still_unfound[13]
print(story['title'])
book_reviews = find_reviews_by_id(new_ids)

# Combine into final record
book = {
    'id': new_ids,
    'title': story['title'],
    'author': story['author'],
    'story': story['text'],
    'av_rating': average_rating,
    'num_ratings':count,
    'reviews': book_reviews
}

data_3.append(book)

The Eagle and the Fox


#### The north wind and the sun

In [214]:
still_unfound[14]['title']

'The North Wind and the Sun'

In [215]:
matches = [s for s in ae if 'sun' in s[0].lower()]
print(matches)

[['The Wind and the Sun', '12728462']]


#### The crab and his mother

In [218]:
still_unfound[15]['title']
matches = [s for s in ae if 'crab' in s[0].lower()]
print(matches)

[['The Crab And Its Mother', '36465981']]


In [52]:
for item in BOOKS:
    if item['book_id'] =='36465981':
        names = []
        for aut_id in item['author_ids']:
            name = find_name_by_id(aut_id)
            names.append(name)
print(names)

['Aesop']


In [53]:
new_ids = ['36465981']
total_ratings = 0
count = 0
for id in new_ids:
    for item in BOOKS:
        if id == item['book_id']:
            try:
                rating_av = float(item['average_rating'])
                num = int(item['ratings_count'])
                total_ratings += rating_av * num
                count += num
            except (ValueError, TypeError, KeyError):
                pass
                    
if count == 0:
    average_rating = 0
else:
    average_rating = total_ratings / count

story = still_unfound[15]
print(story['title'])
book_reviews = find_reviews_by_id(new_ids)

# Combine into final record
book = {
    'id': new_ids,
    'title': story['title'],
    'author': story['author'],
    'story': story['text'],
    'av_rating': average_rating,
    'num_ratings':count,
    'reviews': book_reviews
}

data_3.append(book)

The Crab and His Mother


#### The leopard Man's Story

In [224]:
still_unfound[16]['author']

'Jack London'

In [54]:
jl = find_books_by_author('Jack London')
matches = [s for s in jl if 'leopard' in s[0].lower()]
print(matches)

[["The Leopard Man's Story and Other Stories", '16393802'], ['Die Geschichte vom Leopardenmann', '24977753']]


In [55]:
story = still_unfound[16]
print(story['title'])
new_ids = ['16393802','24977753']
total_ratings = 0
count = 0
for id in new_ids:
    for item in BOOKS:
        if id == item['book_id']:
            try:
                rating_av = float(item['average_rating'])
                num = int(item['ratings_count'])
                total_ratings += rating_av * num
                count += num
            except (ValueError, TypeError, KeyError):
                pass
                    
if count == 0:
    average_rating = 0
else:
    average_rating = total_ratings / count


book_reviews = find_reviews_by_id(new_ids)

# Combine into final record
book = {
    'id': new_ids,
    'title': story['title'],
    'author': story['author'],
    'story': story['text'],
    'av_rating': average_rating,
    'num_ratings':count,
    'reviews': book_reviews
}
data_3.append(book)

The Leopard Man’s Story


#### The miller, his son and their ass

In [231]:
still_unfound[17]['title']
matches = [s for s in ae if 'miller' in s[0].lower()]
print(matches)

[]


#### Mr. and Mrs. Vinegar

In [234]:
still_unfound[18]['title']

'Mr. and Mrs. Vinegar'

In [57]:
fas = find_books_by_author('Flora Annie Steel')
matches = [s for s in fas if 'vinegar' in s[0].lower()]
print(matches)

[]


#### Demir-kayá

In [58]:
still_unfound[19]['title']

'Demir-kayá'

#### The farmer and the fox

In [244]:
still_unfound[20]['title']

'The Farmer and the Fox'

In [246]:
matches = [s for s in ae if 'farmer' in s[0].lower()]
print(matches)

[['The Farmer And The Snake', '32808732']]


#### Come-By-Chance

In [249]:
still_unfound[21]['title']

'Come-By-Chance'

In [60]:
abp = find_books_by_author('Banjo Paterson')
abp.extend(find_books_by_author('A B Paterson'))
abp.extend(find_books_by_author('A.B. Paterson'))
print(abp)
matches = [s for s in abp if 'come' in s[0].lower()]
print(matches)

[['The Merino Sheep', '33391789'], ['Complete Poems', '745346'], ['Twas the Night Before Christmas and Other Christmas Stories', '8468620'], ["We're All Australians Now", '31847636'], ['Poems of Banjo Paterson', '20995192'], ["We're All Australians Now", '24358240'], ['The Cat', '23633435'], ['Waltzing Matilda', '111281'], ['The Best Of Banjo Patterson: An Illustrated Collection', '3172903'], ['The Man from Snowy River', '1841075'], ['Saltbush Bill, J. P', '12255399'], ['The Man From Snowy River', '6072732'], ["The Banjo's Best-Loved Poems", '11480077'], ["Works of 'Banjo' Paterson", '2746647'], ['The Man from Snowy River and Other Verses', '2725644'], ["Mulga Bill's Bicycle", '1651879'], ['The Man from Snowy River', '7902381'], ['An Outback Marriage', '6512349'], ['Clancy Of The Overflow', '397268'], ['Waltzing Matilda', '16074139']]
[]


#### The Laidly Worm

In [254]:
still_unfound[22]['title']

'The Laidly Worm'

In [258]:
matches = [s for s in fas if 'worm' in s[0].lower()]
print(matches)

[]


#### The Boar-Pig

In [262]:
still_unfound[23]['title']

'The Boar-Pig'

In [266]:
# it does exist just can't find it by code 

In [62]:
story = still_unfound[23]

book = {
    'id': [],
    'title': story['title'],
    'author': story['author'],
    'story': story['text'],
    'av_rating': 3.63,
    'num_ratings':38,
    'reviews': [{'rating':4, 'text':"As is to be expected with Saki, the tone is everything, and the conglomeration of details and circumstances most pleasing."},
               {'rating':5,'text':"Another little gem and although it may not have the deliciously anarchic spirit of Saki's really great stories, like 'An Open Window' it is still wonderful, primarily for Matilda, one of the many splendid nieces who populate his stories. It is quite remarkable how often a niece is the agent of the chaos at the heart of Saki's stories, like in 'An Open Window'. Saki is so good on children, he is not a children's' or YA author but they would certainly appreciate his understanding of their devotion to a justice more literal and fierce then you will find in the Old Testament. His children are not 'innocent' in the wakish sense but they recognise and understand adult deceit and hypocrisy. His child characters are observant and understanding of their basic powerlessness but they know how to exploit adult mendacity for their benefit. That the story also punctures adult pomposity and social climbing is an added bonus."}
               ]
}

#print(book)
data_3.append(book)

#### The recovery

In [63]:
still_unfound[24]['title']

'The Recovery'

#### a love by the sea

In [275]:
still_unfound[25]['title']

'A Love by The Sea'

#### The wolf in sheeps clothing

In [276]:
still_unfound[26]

{'title': 'The Wolf in Sheep’s Clothing',
 'author': 'Aesop',
 'text': 'AWolf resolved to disguise himself in order that he might prey upon a flock of sheep without fear of detection. So he clothed himself in a sheepskin, and slipped among the sheep when they were out at pasture. He completely deceived the shepherd, and when the flock was penned for the night he was shut in with the rest. But that very night as it happened, the shepherd, requiring a supply of mutton for the table, laid hands on the Wolf in mistake for a Sheep, and killed him with his knife on the spot.',
 'word_count': 96,
 'url': 'https://www.libraryofshortstories.com/onlinereader/the-wolf-in-sheeps-clothing'}

In [278]:
matches = [s for s in ae if 'wolf' in s[0].lower()]
print(matches)

[['The Domesticated Dog And The Wolf', '36448148'], ['The Wolf And The Lamb', '36437778'], ['The Boy Who Cried Wolf', '1133198'], ['The Town Mouse and the Country Mouse: The Boy Who Cried Wolf', '1275099'], ['The Kid And The Wolf', '33135874'], ["The Wolf in Sheep's Clothing", '24811399'], ['The Wolf And The Lamb', '36465970'], ['The Wolf And The Crane', '33216817']]


In [64]:
story = still_unfound[26]
print(story['title'])
new_ids = ['24811399']
total_ratings = 0
count = 0
for id in new_ids:
    for item in BOOKS:
        if id == item['book_id']:
            try:
                rating_av = float(item['average_rating'])
                num = int(item['ratings_count'])
                total_ratings += rating_av * num
                count += num
            except (ValueError, TypeError, KeyError):
                pass
                    
if count == 0:
    average_rating = 0
else:
    average_rating = total_ratings / count


book_reviews= find_reviews_by_id(new_ids)

# Combine into final record
book = {
    'id': new_ids,
    'title': story['title'],
    'author': story['author'],
    'story': story['text'],
    'av_rating': average_rating,
    'num_ratings':count,
    'reviews': book_reviews
}
#print(book)
data_3.append(book)

The Wolf in Sheep’s Clothing


#### The Last Incantation

In [282]:
still_unfound[27]['title']

'The Last Incantation'

In [65]:
cas = find_books_by_author('Clark Ashton Smith')
matches = [s for s in cas if 'last' in s[0].lower()]
print(matches)

[['The Last Hieroglyph', '5238510'], ['The Last Hieroglyph', '30353483'], ['The Last Oblivion: Best Fantastic Poetry of Clark Ashton Smith', '102085']]


In [286]:
# it exists but can't find it

In [66]:
story = still_unfound[27]
for item in BOOKS:
    if item['title'] == "The Last Incantation":
        print(item['book_id'])
        #new_ids = [item['book_id']]
        #average_rating = item['average_rating']

book = {
    'id': [],
    'title': story['title'],
    'author': story['author'],
    'story': story['text'],
    'av_rating': 3.97,
    'num_reviews':116,
    'reviews': [{'rating':3, 'text':"The story has a deep moral message about the burdens of change. Sometimes we distance ourselves from people and places that once filled us with feelings of joy because we feel like they've changed and they're not as they used to be. However, there are also times when we are the ones that changed without realising it, and people distance themselves away from us without us knowing why we went our separate ways. A harsh truth about growing old."},
               {'rating':5, 'text':" I finally picked up a collection of his stories at Abraxus books, a neat little used bookstore here in Ballard. The first story floored me, and the realization that I had my hands on a potential gem was confirmed by each successive tale. Smith is, first and foremost, a writer's writer. His prose is excellent. It is said that he wrote with an ear for oratory, and I agree: his writing carries a fine sound when read aloud. In comparison with contemporary writers, Smith really shines. He has a lucid style that conveys beautiful imagery in deft, single-sentence strokes. For those readers who, like me, are tired of 'doorstop' fantasy tomes, these tales are a delicious antidote. The tales themselves are packed with wonders and myths and mysteries. Some of them gave me the thrill that only a well-wrought idea can. The fecundity of Smith's imagination is just astounding. Each tale he wrote is a complete, self-contained story, and each contains enough material for a proper novel. Yet Smith never leaves you wanting for more except in the best sense; each tale lingers in the mind with a pleasant echo." }]
}
data_3.append(book)

#### Coming

In [292]:
still_unfound[28]

{'title': 'Coming',
 'author': 'Charlotte Perkins Gilman',
 'text': 'Because the time is ripe, the age is ready,\n\nBecause the world her woman’s help demands,\n\nOut of the long subjection and seclusion\n\nCome to our field of warfare and confusion\n\nThe mother’s heart and hands.\n\nLong has she stood aside, endured and waited,\n\nWhile man swung forward, toiling on alone;\n\nNow, for the weary man, so long ill-mated,\n\nNow, for the world for which she was created,\n\nComes woman to her own.\n\nNot for herself! though sweet the air of freedom;\n\nNot for herself, though dear the new-born power;\n\nBut for the child, who needs a nobler mother,\n\nFor the whole people, needing one another,\n\nComes woman to her hour.',
 'word_count': 111,
 'url': 'https://www.libraryofshortstories.com/onlinereader/coming'}

#### Jim Carew

In [294]:
still_unfound[29]['title']

'Jim Carew'

#### The Feast of Nemesis

In [297]:
still_unfound[30]['title']

'The Feast of Nemesis'

In [300]:
matches = [s for s in saki if 'feast' in s[0].lower()]
print(matches)

[]


#### The Fox and the Crow

In [303]:
still_unfound[31]['title']

'The Fox and the Crow'

In [305]:
matches = [s for s in ae if 'fox' in s[0].lower()]
print(matches)

[['Fox Tails: Four Fables from Aesop', '12710065'], ['The Fox and the Goat', '36437604'], ['The Fox And The Lion', '33131735'], ['The Fox And The Grapes', '33131659'], ['The Bear And The Fox', '36437784'], ['The Eagle And The Fox', '36434944'], ['The Lion And The Fox', '36448128'], ['What The Fox Learnt: Four Fables from Aesop', '12574862'], ['The Dog The Cock And The Fox', '32805954']]


#### The Doer of Good

In [308]:
still_unfound[32]['title']

'The Doer of Good'

In [ ]:
#exists but can't find

In [67]:
story = still_unfound[32]
book = {
    'id': [],
    'title': story['title'],
    'author': story['author'],
    'story': story['text'],
    'av_rating': 3.85,
    'reviews': [{'rating':5, 'text': "A certain unexpected sincerity."},{'rating':5, 'text': "Ooof. OOF. Wilde you magnificent son of a bitch."}]
}
#print(book)
data_3.append(book)

#### The Oracle at the Private Bar

In [314]:
still_unfound[33]['title']

'The Oracle at the Private Bar'

#### The boy and the filberts

In [319]:
still_unfound[34]['title']

'The Boy and the Filberts'

In [320]:
matches = [s for s in ae if 'boy' in s[0].lower()]
print(matches)

[['The Boy Who Cried Wolf', '1133198'], ['The Town Mouse and the Country Mouse: The Boy Who Cried Wolf', '1275099'], ['The Boy And The Scorpion', '36434972']]


#### Lost

In [322]:
still_unfound[35]['title']

'Lost'

#### She walketh veiled and sleeping

In [324]:
still_unfound[36]

{'title': 'She Walketh Veiled and Sleeping',
 'author': 'Charlotte Perkins Gilman',
 'text': 'She walketh veiled and sleeping,\n\nFor she knoweth not her power;\n\nShe obeyeth but the pleading\n\nOf her heart, and the high leading\n\nOf her soul, unto this hour.\n\nSlow advancing, halting, creeping,\n\nComes the Woman to the hour!—\n\nShe walketh veiled and sleeping,\n\nFor she knoweth not her power.',
 'word_count': 50,
 'url': 'https://www.libraryofshortstories.com/onlinereader/she-walketh-veiled-and-sleeping'}

#### A strange Story

In [68]:
still_unfound[37]['title']

story = still_unfound[37]
book = {
    'id': [],
    'title': story['title'],
    'author': story['author'],
    'story': story['text'],
    'av_rating': 3.03,
    'reviews': [{'rating':3, 'text': "This was a well named story. I like to picture the father outside, with comically intense OCD, for years, waiting for the right moment to go inside. That may or may not be funny to some people."},
                {'rating':3, 'text': "Yup, strange indeed yet I loved it"},
               {'rating':3, 'text':"Had it been a novel, it would have made for an interesting read for sure. The more I read O' Henry, the more I feel that somehow, this man tested his potential in briefing things. A girl's father missing and then returning to his home after years, that too under the same circumstances, though for a different person this time; in two pages, this man has wrapped up a novel of almost 150 pages. Wow! Some talent. This short read was good."}]
}
#print(book)
data_3.append(book)

#### the mice and the weasels

In [331]:
still_unfound[38]['title']

'The Mice and the Weasels'

In [335]:
matches = [s for s in ae if 'mice' in s[0].lower()]
print(matches)

[]


#### The Amateur Rider

In [70]:
still_unfound[39]['title']

'The Amateur Rider'

In [71]:
print(abp)

[['The Merino Sheep', '33391789'], ['Complete Poems', '745346'], ['Twas the Night Before Christmas and Other Christmas Stories', '8468620'], ["We're All Australians Now", '31847636'], ['Poems of Banjo Paterson', '20995192'], ["We're All Australians Now", '24358240'], ['The Cat', '23633435'], ['Waltzing Matilda', '111281'], ['The Best Of Banjo Patterson: An Illustrated Collection', '3172903'], ['The Man from Snowy River', '1841075'], ['Saltbush Bill, J. P', '12255399'], ['The Man From Snowy River', '6072732'], ["The Banjo's Best-Loved Poems", '11480077'], ["Works of 'Banjo' Paterson", '2746647'], ['The Man from Snowy River and Other Verses', '2725644'], ["Mulga Bill's Bicycle", '1651879'], ['The Man from Snowy River', '7902381'], ['An Outback Marriage', '6512349'], ['Clancy Of The Overflow', '397268'], ['Waltzing Matilda', '16074139']]


#### Requiescat

In [340]:
still_unfound[40]['title']

'Requiescat'

In [72]:
story = still_unfound[40]
book = {
    'id': [],
    'title': story['title'],
    'author': story['author'],
    'story': story['text'],
    'av_rating': 3.03,
    'reviews': [{'rating':5, 'text': "'Tread lightly, she is near' this was my instagram bio when i was sixteen because i love this poem so much. I’m not big on poetry, but this one feels like waking up on a crystal winter morning. very calm and serene."}]
}
#print(book)
data_3.append(book)

#### Saltbush Bill

In [345]:
still_unfound[41]['title']

'Saltbush Bill'

In [73]:
print(abp)

[['The Merino Sheep', '33391789'], ['Complete Poems', '745346'], ['Twas the Night Before Christmas and Other Christmas Stories', '8468620'], ["We're All Australians Now", '31847636'], ['Poems of Banjo Paterson', '20995192'], ["We're All Australians Now", '24358240'], ['The Cat', '23633435'], ['Waltzing Matilda', '111281'], ['The Best Of Banjo Patterson: An Illustrated Collection', '3172903'], ['The Man from Snowy River', '1841075'], ['Saltbush Bill, J. P', '12255399'], ['The Man From Snowy River', '6072732'], ["The Banjo's Best-Loved Poems", '11480077'], ["Works of 'Banjo' Paterson", '2746647'], ['The Man from Snowy River and Other Verses', '2725644'], ["Mulga Bill's Bicycle", '1651879'], ['The Man from Snowy River', '7902381'], ['An Outback Marriage', '6512349'], ['Clancy Of The Overflow', '397268'], ['Waltzing Matilda', '16074139']]


In [74]:
story = still_unfound[41]
print(story['title'])
new_ids = ['12255399']
total_ratings = 0
count = 0
for id in new_ids:
    for item in BOOKS:
        if id == item['book_id']:
            try:
                rating_av = float(item['average_rating'])
                num = int(item['ratings_count'])
                total_ratings += rating_av * num
                count += num
            except (ValueError, TypeError, KeyError):
                pass
                    
if count == 0:
    average_rating = 0
else:
    average_rating = total_ratings / count


book_reviews = find_reviews_by_id(new_ids)

# Combine into final record
book = {
    'id': new_ids,
    'title': story['title'],
    'author': story['author'],
    'story': story['text'],
    'av_rating': average_rating,
    'num_reviews':count,
    'reviews': book_reviews
}
#print(book)
data_3.append(book)

Saltbush Bill


#### The rose tree

In [352]:
still_unfound[42]['title']

'The Rose Tree'

In [357]:
matches = [s for s in fas if 'tree' in s[0].lower()]
print(matches)

[]


#### Father and sons

In [359]:
still_unfound[43]['title']

'Father and Sons'

In [360]:
matches = [s for s in ae if 'son' in s[0].lower()]
print(matches)

[['Lessons from the Lion, the Ox and their little friends', '13556167'], ['Lessons from the Lion, the Ox and their little friends (illustrated)', '18601144'], ['Iliad, Odyssey, Aeneid, Oedipus, Jason and the Argonauts and 50+ Legendary Books', '12091515']]


#### A Legend

In [363]:
still_unfound[44]['title']

'A Legend'

In [366]:
aik = find_books_by_author('Aleksandr I. Kuprin')
print(aik)

[]


In [75]:
aik = find_books_by_author('Aleksandr Kuprin')
print(aik)

[]


#### Lawkamercyme

In [369]:
still_unfound[45]['title']

'Lawkamercyme'

In [373]:
matches = [s for s in fas if 'law' in s[0].lower()]
print(matches)

[]


#### Steelman's Pupil

In [376]:
still_unfound[46]['title']

'Steelman’s Pupil'

In [379]:
hl = find_BOOKS_by_author('Henry Lawson')
matches = [s for s in hl if 'steel' in s[0].lower()]
print(matches)

[]


#### The dog and the shadow

In [381]:
still_unfound[47]['title']

'The Dog and the Shadow'

In [76]:
matches = [s for s in ae if 'dog' in s[0].lower()]
print(matches)

[['The Domesticated Dog And The Wolf', '36448148'], ['The Dog And The Shadow', '36454743'], ['The Dog The Cock And The Fox', '32805954']]


In [78]:
story = still_unfound[47]
print(story['title'])
new_ids = ['36454743']
total_ratings = 0
count = 0
for id in new_ids:
    for item in BOOKS:
        if id == item['book_id']:
            try:
                rating_av = float(item['average_rating'])
                num = int(item['ratings_count'])
                total_ratings += rating_av * num
                count += num
            except (ValueError, TypeError, KeyError):
                pass
                    
if count == 0:
    average_rating = 0
else:
    average_rating = total_ratings / count


book_reviews = find_reviews_by_id(new_ids)

# Combine into final record
book = {
    'id': new_ids,
    'title': story['title'],
    'author': story['author'],
    'story': story['text'],
    'av_rating': average_rating,
    'num_ratings':count,
    'reviews': book_reviews
}
#print(book)
data_3.append(book)

The Dog and the Shadow


#### How the whale got his throat

In [386]:
still_unfound[48]['title']

'How The Whale Got His Throat'

In [80]:
rk = find_books_by_author('Rudyard Kipling')
matches = [s for s in rk if 'whale' in s[0].lower()]
print(matches)

[['How the Whale Got His Throat', '1548238']]


In [392]:
# not actually this text, has been retold, but there is another one 

In [81]:
story = still_unfound[48]
book = {
    'id': [],
    'title': story['title'],
    'author': story['author'],
    'story': story['text'],
    'av_rating': 3.00,
    'reviews': [{'rating':3,'text':"A very amusing story by Kipling."}]
}
#print(book)
data_3.append(book)

#### the story of a mouse that helped

In [396]:
still_unfound[49]['title']

'Filboid Studge, The Story of a Mouse that Helped'

In [397]:
matches = [s for s in saki if 'mouse' in s[0].lower()]
print(matches)

[['The Mouse', '21567585'], ['The Open Window & The Mouse', '8744313'], ['Filboid Studge, The Story of a Mouse That Helped', '10548737'], ['The Mouse', '10548738']]


In [82]:
story = still_unfound[49]
print(story['title'])
new_ids = ['10548737']
total_ratings = 0
count = 0
for id in new_ids:
    for item in BOOKS:
        if id == item['book_id']:
            try:
                rating_av = float(item['average_rating'])
                num = int(item['ratings_count'])
                total_ratings += rating_av * num
                count += num
            except (ValueError, TypeError, KeyError):
                pass
                    
if count == 0:
    average_rating = 0
else:
    average_rating = total_ratings / count

book_reviews = find_reviews_by_id(new_ids)

# Combine into final record
book = {
    'id': new_ids,
    'title': story['title'],
    'author': story['author'],
    'story': story['text'],
    'av_rating': average_rating,
    'num_ratings':count,
    'reviews': book_reviews
}
#print(book)
data_3.append(book)

Filboid Studge, The Story of a Mouse that Helped


#### The emperors new clothes

In [401]:
still_unfound[50]['title']

'The Emperor’s New Clothes'

In [83]:
hca = find_books_by_author('Hans Christian Andersen')
matches = [s for s in hca if 'new' in s[0].lower()]
print(matches)

[["The Emperor's New Clothes", '343002'], ["The Emperor's New Clothes", '343007'], ["The Emperor's New Clothes", '18484340'], ["The emperor's new clothes", '8460445'], ["Pakaian Baru Kaisar = Emperor's New Clothes", '10339960'], ["The Emperor's New Clothes", '5954303'], ["Andersen's Fairy Tales: The Emperor's New Clothes/The Ugly Duckling/The Little Match Girl/Big Claus and Little Claus and Many More", '6912928'], ["The Emperor's new clothes and two other stories", '6411029'], ["The Emperor's New Clothes", '343000'], ["The Emperor's New Clothes", '152465'], ["The Emperor's New Clothes", '1611857'], ["The Emperor's New Clothes: The Graphic Novel", '6702028'], ["The Emperor's New Clothes", '176774'], ['The Stories of Hans Christian Andersen: A New Translation from the Danish', '1123513'], ["The Emperor's New Clothes", '6124108'], ["The Emperor's New Clothes", '1094089'], ["The Dinosaur's New Clothes", '1201068'], ["Hans Christian Andersen's the Emperor's New Clothes", '6729778'], ["The E

In [84]:
names = []
id_ = '6687866'
for item in BOOKS:
    if item['book_id'] == id_:
        for aut_id in item['author_ids']:
            name = find_name_by_id(aut_id)
            if name:
                names.append(name)
        print(names, id_)

['Dorothee Duntze', 'Anthea Bell', 'Hans Christian Andersen'] 6687866


In [88]:
new_ids = ['343002','18484340','5954303','1094089']
story = still_unfound[50]
print(story['title'])
total_ratings = 0
count = 0
for id in new_ids:
    for item in BOOKS:
        if id == item['book_id']:
            try:
                rating_av = float(item['average_rating'])
                num = int(item['ratings_count'])
                total_ratings += rating_av * num
                count += num
            except (ValueError, TypeError, KeyError):
                pass
                    
if count == 0:
    average_rating = 0
else:
    average_rating = total_ratings / count


book_reviews = find_reviews_by_id(new_ids)

# Combine into final record
book = {
    'id': new_ids,
    'title': story['title'],
    'author': story['author'],
    'story': story['text'],
    'av_rating': average_rating,
    'num_ratings':count,
    'reviews': book_reviews
}
#print(book)
data_3.append(book)

The Emperor’s New Clothes


#### The last parade

In [644]:
still_unfound[51]['title']

'The Last Parade'

#### Chiming a Dream

In [440]:
still_unfound[52]['title']

'Chiming a Dream'

In [439]:
find_BOOKS_by_author('William Ernest Henley', BOOKS, AUTHORS)

[['One Hundred and One Famous Poems eith a Prose Supplement', '5956892'],
 ['One Hundred and One Famous Poems: With a Prose Supplement', '5956903'],
 ['Hawthorn and Lavender', '2486948'],
 ['One Hundred and One Famous Poems : With a Prose Supplement', '933483'],
 ['101 Famous Poems', '933482'],
 ['101 Famous Poems', '933481'],
 ['One Hundred and One Famous Poems: With a Prose Supplement', '304728'],
 ['101 Famous Poems: With a Prose Supplement', '11639267'],
 ['Beaten but Unbowed: Waking From The Nightmare Of Abuse', '34843589'],
 ['One Hundred And One Famous Poems', '1221921'],
 ['Invictus', '9465133'],
 ['Invictus', '20523478'],
 ['One Hundred And One Famous Poems', '1124927'],
 ['English Victorian Poetry: An Anthology', '729036'],
 ['The World Treasury of Modern Religious Thought', '763617']]

#### the Frog prince

In [442]:
still_unfound[53]['title']

'The Frog-Prince'

In [89]:
names = []
for item in BOOKS:
    if 'Frog Prince' in item['title']:
        names = []
        for aut_id in item['author_ids']:
            name = find_name_by_id(aut_id)
            if name:
                names.append(name)
        print(item['book_id'], item['title'], names)

6383800 Dragon Kiss (The Tales of the Frog Princess, #7) ['E.D. Baker']
1118701 The Frog Prince ['Susanna Davidson']
2174173 The Frog Prince, ['Jacob Grimm']
28219800 Grimms Storytime Library Boxed Set (The Frog Prince,Rumpelstiltskin,The Brave Little Tailor,Snow White,Rapunzel) ['Robyn Bryant']
9294411 The Frog Prince ['Elle Lothlorien']
23936658 The Frog Prince ['Sophie Ranald']
33607137 The Frog Prince: The Brothers Grimm Story Told as a Novella ['Mike Klaassen']
137078 The Frog Prince ['Kathy-jo Wargin', 'Anne Yvonne Gilbert']
137077 The Frog Prince ['Jan Ormerod']
137075 The Frog Prince (We Both Read - Level 1-2) ['Sindy McKay', 'Jacob Grimm']
137072 The Frog Prince ['Jane Porter']
137073 The Frog Prince ['Edith H. Tarcov', 'James  Marshall', 'Edith H. Tarcov']
1634608 The Frog Prince ['Hilary Robinson']
17657680 Frankly, I Never Wanted to Kiss Anybody!: The Story of the Frog Prince as Told by the Frog ['Nancy Loewen', 'Denis Alonso']
887501 The Frog Prince (To Catch a Prince, #2)

In [91]:
new_ids = ['2174173','21353471']
story = still_unfound[53]
print(story['title'])
total_ratings = 0
count = 0
for id in new_ids:
    for item in BOOKS:
        if id == item['book_id']:
            try:
                rating_av = float(item['average_rating'])
                num = int(item['ratings_count'])
                total_ratings += rating_av * num
                count += num
            except (ValueError, TypeError, KeyError):
                pass
                    
if count == 0:
    average_rating = 0
else:
    average_rating = total_ratings / count


book_reviews = find_reviews_by_id(new_ids)

# Combine into final record
book = {
    'id': new_ids,
    'title': story['title'],
    'author': story['author'],
    'story': story['text'],
    'av_rating': average_rating,
    'num_ratings':count,
    'reviews': book_reviews
}
#print(book)
data_3.append(book)

The Frog-Prince


#### Under the Shadow of Kiley’s Hill

In [460]:
still_unfound[54]['title']

'Under the Shadow of Kiley’s Hill'

#### In the Forest of Villefère

In [462]:
still_unfound[55]['title']

'In the Forest of Villefère'

In [92]:
reh = find_books_by_author('Robert E. Howard')
matches = [s for s in reh if 'forest' in s[0].lower()]
print(matches)

[['In The Forest Of Villefere', '7749021']]


In [94]:
new_ids = ['7749021']
story = still_unfound[55]
print(story['title'])
total_ratings = 0
count = 0
for id in new_ids:
    for item in BOOKS:
        if id == item['book_id']:
            try:
                rating_av = float(item['average_rating'])
                num = int(item['ratings_count'])
                total_ratings += rating_av * num
                count += num
            except (ValueError, TypeError, KeyError):
                pass
                    
if count == 0:
    average_rating = 0
else:
    average_rating = total_ratings / count


book_reviews = find_reviews_by_id(new_ids)

# Combine into final record
book = {
    'id': new_ids,
    'title': story['title'],
    'author': story['author'],
    'story': story['text'],
    'av_rating': average_rating,
    'num_ratings':count,
    'reviews': book_reviews
}
#print(book)
data_3.append(book)

In the Forest of Villefère


#### The thief

In [468]:
still_unfound[56]['title']

'The Thief'

In [470]:
find_BOOKS_by_author('Rene Maizeroy')

[]

#### A Vision of Judgment

In [642]:
still_unfound[57]['title']

'A Vision of Judgment'

In [95]:
for item in BOOKS:
    if item['title'] == 'A Vision of Judgment':
        print(item['book_id'])

In [96]:
hgw =find_books_by_author('H.G. Wells')
matches = [s for s in hgw if 'vision' in s[0].lower()]
print(matches)

[['A Vision Of Judgment', '23355888'], ['Visions of Tomorrow: Science Fiction Predictions that Came True', '8387459'], ['Fantastic Tales: Visionary and Everyday', '135034']]


In [98]:
new_ids = ['23355888']
story = still_unfound[57]
print(story['title'])
total_ratings = 0
count = 0
for id in new_ids:
    for item in BOOKS:
        if id == item['book_id']:
            try:
                rating_av = float(item['average_rating'])
                num = int(item['ratings_count'])
                total_ratings += rating_av * num
                count += num
            except (ValueError, TypeError, KeyError):
                pass
                    
if count == 0:
    average_rating = 0
else:
    average_rating = total_ratings / count


book_reviews = find_reviews_by_id(new_ids)

# Combine into final record
book = {
    'id': new_ids,
    'title': story['title'],
    'author': story['author'],
    'story': story['text'],
    'av_rating': average_rating,
    'num_ratings':count,
    'reviews': book_reviews
}
#print(book)
data_3.append(book)

A Vision of Judgment


#### In Defence of the Bush'

In [481]:
still_unfound[58]['title']

'In Defence of the Bush'

#### Work, Death and Sickness

In [483]:
still_unfound[59]['title']

'Work, Death and Sickness'

In [99]:
lt =find_books_by_author('Leo Tolstoy')
matches = [s for s in lt if 'sick' in s[0].lower()]
print(matches)

[]


In [100]:
story = still_unfound[59]
book = {
    'id': new_ids,
    'title': story['title'],
    'author': story['author'],
    'story': story['text'],
    'av_rating': 3.4,
    'num_ratings':110,
    'reviews': [{'rating':5,'text':"A simple story that tells how we should live..."},
               {'rating':4,'text':"God works under trial and error"},
               {'rating':3, 'text':"This is a cool and concise look at man’s purpose on earth. I enjoyed it."},
               {'rating':4, 'text':"Leo Tolstoy presented a fascinating and profound insight into the complexities of human societies."}]
}
#print(book)
data_3.append(book)

#### The old woman and her pig

In [496]:
still_unfound[60]['title']

'The Old Woman and Her Pig'

In [497]:
matches = [s for s in fas if 'pig' in s[0].lower()]
print(matches)

[]


#### For all we have and are

In [500]:
still_unfound[61]['title']

'“For All We Have And Are”'

In [504]:
matches = [s for s in rk if 'have' in s[0].lower()]
print(matches)

[]


In [101]:
for item in BOOKS:
    if item['title'] == 'For All We Have And Are':
        print(item['book_id'])

#### The fox and the stork

In [507]:
still_unfound[62]['title']

'The Fox and the Stork'

In [509]:
matches = [s for s in ae if 'stork' in s[0].lower()]
print(matches)

[]


In [102]:
for item in BOOKS:
    if item['title'] == 'The Fox and the Stork':
        names = []
        for aut_id in item['author_ids']:
            name = find_name_by_id(aut_id)
            if name:
                names.append(name)
        print(item['book_id'], item['title'], names)

2546793 The Fox and the Stork ['Mairi Mackinnon', 'Rocio Martinez']
35287270 The Fox and the Stork ['Beatrix Potter']
3962938 The Fox and the Stork ['Gerald McDermott']


In [103]:
story = still_unfound[62]
book = {
    'id': new_ids,
    'title': story['title'],
    'author': story['author'],
    'story': story['text'],
    'av_rating': 3.68,
    'num_ratings':22,
    'reviews': [{'rating':5,'text':"'Do not play tricks on your neighbors unless you can stand the same treatment yourself.'"}]
}
#print(book)
data_3.append(book)

#### On kiley's run

In [517]:
still_unfound[63]['title']

'On Kiley’s Run'

#### The relic

In [519]:
still_unfound[64]['title']

'The Relic'

#### the story-teller

In [521]:
still_unfound[65]['title']

'The Story-Teller'

In [522]:
matches = [s for s in saki if 'story' in s[0].lower()]
print(matches)

[['Short Story Masterpieces', '259826'], ['The Sixth Ghost Story MEGAPACK: 25 Classic Ghost Stories', '25414261'], ['Short Story Masterpieces', '8643862'], ['The Storyteller', '27866029'], ['Filboid Studge, The Story of a Mouse That Helped', '10548737'], ['The Story-Teller: Thirteen Tales', '2573028'], ['Librivox Short Story Collection 029 (Librivox Short Stories, #29)', '22041396'], ['Horror Story Collection 001', '24131064']]


In [104]:
new_ids = ['27866029']
story = still_unfound[65]
print(story['title'])
total_ratings = 0
count = 0
for id in new_ids:
    for item in BOOKS:
        if id == item['book_id']:
            try:
                rating_av = float(item['average_rating'])
                num = int(item['ratings_count'])
                total_ratings += rating_av * num
                count += num
            except (ValueError, TypeError, KeyError):
                pass
                    
if count == 0:
    average_rating = 0
else:
    average_rating = total_ratings / count


book_reviews = find_reviews_by_id(new_ids)

# Combine into final record
book = {
    'id': new_ids,
    'title': story['title'],
    'author': story['author'],
    'story': story['text'],
    'av_rating': average_rating,
    'num_ratings':count,
    'reviews': book_reviews
}
#print(book)
data_3.append(book)

The Story-Teller


#### The frogs asking for a king

In [532]:
still_unfound[66]['title']

'The Frogs Asking for a King'

In [536]:
matches = [s for s in ae if 'king' in s[0].lower()]
print(matches)

[['The Creaking Wheels', '36448111']]


#### the two sisters

In [538]:
still_unfound[67]['title']

'The Two Sisters'

#### A Child’s Dream of a Star

In [540]:
still_unfound[68]['title']

'A Child’s Dream of a Star'

In [543]:
matches = [s for s in chls_dic_bks if 'star' in s[0].lower()]
print(matches)

[["A Child's Dream of a Star", '10975557'], ["A Child's Dream of a Star", '11271813'], ['Great Expectations (Classic Starts Series)', '6856992']]


In [105]:
new_ids = ['10975557','11271813']
story = still_unfound[68]
print(story['title'])
total_ratings = 0
count = 0
for id in new_ids:
    for item in BOOKS:
        if id == item['book_id']:
            try:
                rating_av = float(item['average_rating'])
                num = int(item['ratings_count'])
                total_ratings += rating_av * num
                count += num
            except (ValueError, TypeError, KeyError):
                pass
                    
if count == 0:
    average_rating = 0
else:
    average_rating = total_ratings / count


book_reviews = find_reviews_by_id(new_ids)

# Combine into final record
book = {
    'id': new_ids,
    'title': story['title'],
    'author': story['author'],
    'story': story['text'],
    'av_rating': average_rating,
    'num_ratings':count,
    'reviews': book_reviews
}
#print(book)
data_3.append(book)

A Child’s Dream of a Star


#### 'How Gilbert Died'

In [547]:
still_unfound[69]['title']

'How Gilbert Died'

#### How the Camel got his hump

In [549]:
still_unfound[70]['title']

'How The Camel Got His Hump'

In [550]:
matches = [s for s in rk if 'hump' in s[0].lower()]
print(matches)

[['How the Camel Got His Hump', '3770446'], ['How The Camel Got His Hump (Just So Story Series)', '2572894'], ['How the Camel Got His Hump', '2572864'], ['How the Camel Got His Hump', '918375']]


In [106]:
names = []
for item in BOOKS:
    if item['book_id'] == '918375':
        names = []
        for aut_id in item['author_ids']:
            name = find_name_by_id(aut_id)
            if name:
                names.append(name)
        print(item['book_id'], item['title'], names)

918375 How the Camel Got His Hump ['Rudyard Kipling']


In [107]:
find_reviews_by_id(['918375'])

[{'rating': 2,
  'text': "This was boring. My students didn't understand what was going on and they didn't enjoy the illustrations of this book."}]

In [108]:
new_ids = ['3770446','918375']
story = still_unfound[70]
print(story['title'])
total_ratings = 0
count = 0
for id in new_ids:
    for item in BOOKS:
        if id == item['book_id']:
            try:
                rating_av = float(item['average_rating'])
                num = int(item['ratings_count'])
                total_ratings += rating_av * num
                count += num
            except (ValueError, TypeError, KeyError):
                pass
                    
if count == 0:
    average_rating = 0
else:
    average_rating = total_ratings / count


book_reviews = find_reviews_by_id(new_ids)
book_reviews.append({'rating':2, 'text':"I didn't like this story as much as a child...I thought it was unfair that the camel should be forced to work for Man if he didn't want to. I guess I always had an egalitarian view of the animal world."})
book_reviews.append({'rating':3, 'text':"Well, I had to read this (extremely short) short story for an NGO that I had volunteered for. Story is not great, I didn't like the way it is written either. Maybe, kids might like it? Anyway, I liked the poem in the end."})

# Combine into final record
book = {
    'id': new_ids,
    'title': story['title'],
    'author': story['author'],
    'story': story['text'],
    'av_rating': average_rating,
    'num_ratings':count,
    'reviews': book_reviews
}
#print(book)
data_3.append(book)

How The Camel Got His Hump


#### 'Wit Inspirations of the “Two-Year-Olds”

In [561]:
still_unfound[71]['title']

'Wit Inspirations of the “Two-Year-Olds”'

In [565]:
mt = find_BOOKS_by_author('Mark Twain',BOOKS,AUTHORS)
matches = [s for s in rk if 'year' in s[0].lower()]
print(matches)

[['The Years Between', '6640159']]


#### Faces inthe street

In [567]:
still_unfound[72]['title']

'Faces in the Street'

In [570]:
matches = [s for s in hl if 'face' in s[0].lower()]
print(matches)

[]


#### The Cobbler Turned Doctor

In [572]:
still_unfound[73]['title']

'The Cobbler Turned Doctor'

In [575]:
matches = [s for s in ae if 'cobbler' in s[0].lower()]
print(matches)

[]


#### Magnetism

In [110]:
still_unfound[74]['title']

'Magnetism'

In [111]:
story = still_unfound[74]
book = {
    'id': new_ids,
    'title': story['title'],
    'author': story['author'],
    'story': story['text'],
    'av_rating': 3.29,
    'num_ratings':45,
    'reviews': [{'rating':3,'text':"One who doubts telepathy (magnetism) is one who has never experienced the level of emotion necessary for its transmission or receipt."},
               {'rating':3,'text':"3,5"}]
}
#print(book)
data_3.append(book)

#### the milkmaide and her pail

In [581]:
still_unfound[75]['title']

'The Milkmaid and Her Pail'

In [583]:
matches = [s for s in ae if 'milk' in s[0].lower()]
print(matches)

[]


#### An Astrologer’s Song

In [585]:
still_unfound[76]['title']

'An Astrologer’s Song'

In [588]:
matches = [s for s in rk if 'song' in s[0].lower()]
print(matches)

[['A Song of the English (1912)', '6265380']]


#### The farmer and his sons

In [590]:
still_unfound[77]['title']

'The Farmer and His Sons'

In [592]:
matches = [s for s in ae if 'farmer' in s[0].lower()]
print(matches)

[['The Farmer And The Snake', '32808732']]


#### the shirt collar

In [594]:
still_unfound[78]['title']

'The Shirt-Collar'

In [596]:
matches = [s for s in hca if 'shirt' in s[0].lower()]
print(matches)

[['The Shirt Collar', '12078265']]


In [112]:
new_ids = ['12078265']
story = still_unfound[78]
print(story['title'])
total_ratings = 0
count = 0
for id in new_ids:
    for item in BOOKS:
        if id == item['book_id']:
            try:
                rating_av = float(item['average_rating'])
                num = int(item['ratings_count'])
                total_ratings += rating_av * num
                count += num
            except (ValueError, TypeError, KeyError):
                pass
                    
if count == 0:
    average_rating = 0
else:
    average_rating = total_ratings / count


book_reviews = find_reviews_by_id(new_ids)

# Combine into final record
book = {
    'id': new_ids,
    'title': story['title'],
    'author': story['author'],
    'story': story['text'],
    'av_rating': average_rating,
    'num_ratings':count,
    'reviews': book_reviews
}
#print(book)
data_3.append(book)

The Shirt-Collar


#### The Oracle at the Races

In [601]:
still_unfound[79]['title']

'The Oracle at the Races'

#### locked Inside

In [603]:
still_unfound[80]['title']

'Locked Inside'

#### A  Voice from the Town

In [605]:
still_unfound[81]['title']

'A Voice from the Town'

#### How The Rhinocerous Got His Skin'

In [607]:
still_unfound[82]['title']

'How The Rhinocerous Got His Skin'

In [608]:
matches = [s for s in rk if 'skin' in s[0].lower()]
print(matches)

[['How the Rhinoceros Got His Skin', '181491'], ['How The Rhinoceros Got His Skin', '7050817'], ['How the Rhinoceros Got His Skin: The Graphic Novel', '12463323'], ['How The Rhinoceros Got His Skin', '10682618']]


In [113]:
names = []
for item in BOOKS:
    if item['book_id'] == '10682618':
        names = []
        for aut_id in item['author_ids']:
            name = find_name_by_id(aut_id,)
            if name:
                names.append(name)
        print(item['book_id'], item['title'], names)

10682618 How The Rhinoceros Got His Skin ['Heather Adams', 'Rudyard Kipling']


#### How Jack Went Out To Seek His Fortune

In [613]:
still_unfound[83]['title']

'How Jack Went Out To Seek His Fortune'

In [619]:
matches = [s for s in fas if 'jack' in s[0].lower()]
print(matches)

[]


#### The two devines

In [621]:
still_unfound[84]['title']

'The Two Devines'

#### The Birds, the Beasts, and the Bat

In [623]:
still_unfound[85]['title']

'The Birds, the Beasts, and the Bat'

In [624]:
matches = [s for s in ae if 'bat' in s[0].lower()]
print(matches)

[]


#### A vendetta

In [627]:
still_unfound[86]['title']

'A Vendetta'

In [628]:
for item in BOOKS:
    if item['title'] == 'Une Vendetta':
        print(item['book_id'])

In [115]:
gdm = find_books_by_author('Guy de Maupassant',)
matches = [s for s in gdm if 'vendetta' in s[0].lower()]
print(matches)

[['La Vendetta', '6270962']]


In [116]:
find_reviews_by_id(['6270962'],)

[{'rating': 4,
  'text': 'This story takes you down a fairly dark corridor if you stop to think about it. My haiku doesn\'t do it justice, but I\'ve arranged my thoughts into one all the same: \n "Internalizing \n Pangs of hunger and remorse, \n In the name of love."'},
 {'rating': 4, 'text': ''}]

In [117]:
new_ids = ['6270962']
story = still_unfound[86]
print(story['title'])
total_ratings = 0
count = 0
for id in new_ids:
    for item in BOOKS:
        if id == item['book_id']:
            try:
                rating_av = float(item['average_rating'])
                num = int(item['ratings_count'])
                total_ratings += rating_av * num
                count += num
            except (ValueError, TypeError, KeyError):
                pass
                    
if count == 0:
    average_rating = 0
else:
    average_rating = total_ratings / count


book_reviews = find_reviews_by_id(new_ids,)

# Combine into final record
book = {
    'id': new_ids,
    'title': story['title'],
    'author': story['author'],
    'story': story['text'],
    'av_rating': average_rating,
    'num_ratings':count,
    'reviews': book_reviews
}
#print(book)
data_3.append(book)

A Vendetta


#### The Bogey-Beast

In [640]:
still_unfound[87]['title']

'The Bogey-Beast'

#### Between the Dusk of a Summer Night

In [647]:
still_unfound[88]#['title']

{'title': 'Between the Dusk of a Summer Night',
 'author': 'William Ernest Henley',
 'text': 'Between the dusk of a summer night\n\nAnd the dawn of a summer day,\n\nWe caught at a mood as it passed in flight,\n\nAnd we bade it stoop and stay.\n\nAnd what with the dawn of night began\n\nWith the dusk of day was done;\n\nFor that is the way of woman and man,\n\nWhen a hazard has made them one.\n\nArc upon arc, from shade to shine,\n\nThe World went thundering free;\n\nAnd what was his errand but hers and mine —\n\nThe lords of him, I and she?\n\nO, it’s die we must, but it’s live we can,\n\nAnd the marvel of earth and sun\n\nIs all for the joy of woman and man\n\nAnd the longing that makes them one.',
 'word_count': 124,
 'url': 'https://www.libraryofshortstories.com/onlinereader/between-the-dusk-of-a-summer-night'}

In [654]:
weh = find_BOOKS_by_author('William Ernest Henley', BOOKS, AUTHORS)
matches = [s for s in weh if 'dusk' in s[0].lower()]
print(matches)

[]


#### The bee and jupiter

In [656]:
still_unfound[89]['title']

'The Bee and Jupiter'

In [657]:
matches = [s for s in ae if 'bee' in s[0].lower()]
print(matches)

[]


#### Tje clockmaker

In [660]:
still_unfound[90]['title']

'The Clockmaker'

In [659]:
rls = find_BOOKS_by_author('Robert Louis Stevenson', BOOKS, AUTHORS)
matches = [s for s in rls if 'clock' in s[0].lower()]
print(matches)

[]


#### Those names

In [662]:
still_unfound[91]['title']

'Those Names'

#### the winds message

In [664]:
still_unfound[92]['title']

'The Wind’s Message'

#### ambition and art

In [666]:
still_unfound[93]['title']

'Ambition and Art'

#### A mother of monsters

In [668]:
still_unfound[94]['title']

'A Mother of Monsters'

In [669]:
matches = [s for s in gdm if 'mother' in s[0].lower()]
print(matches)

[['A Mother of Monsters:', '22528441']]


In [118]:
new_ids = ['22528441']
story = still_unfound[94]
print(story['title'])
total_ratings = 0
count = 0
for id in new_ids:
    for item in BOOKS:
        if id == item['book_id']:
            try:
                rating_av = float(item['average_rating'])
                num = int(item['ratings_count'])
                total_ratings += rating_av * num
                count += num
            except (ValueError, TypeError, KeyError):
                pass
                    
if count == 0:
    average_rating = 0
else:
    average_rating = total_ratings / count


book_reviews = find_reviews_by_id(new_ids,)

# Combine into final record
book = {
    'id': new_ids,
    'title': story['title'],
    'author': story['author'],
    'story': story['text'],
    'av_rating': average_rating,
    'num_ratings':count,
    'reviews': book_reviews
}
#print(book)
data_3.append(book)

A Mother of Monsters


#### Life is bitter 

In [673]:
still_unfound[95]['title']

'Life is Bitter'

In [676]:
matches = [s for s in weh if 'life' in s[0].lower()]
print(matches)

[]


#### justice

In [678]:
still_unfound[96]['title']

'Justice'

In [681]:
matches = [s for s in rk if 'justice' in s[0].lower()]
print(matches)

[]


#### the history of the necronomicon

In [683]:
still_unfound[97]['title']

'The History of the Necronomicon'

In [119]:
for item in BOOKS:
    if item['title'] == 'History of the Necronomicon':
        print(item['book_id'])

324636


In [120]:
new_ids = ['324636']
story = still_unfound[97]
print(story['title'])
total_ratings = 0
count = 0
for id in new_ids:
    for item in BOOKS:
        if id == item['book_id']:
            try:
                rating_av = float(item['average_rating'])
                num = int(item['ratings_count'])
                total_ratings += rating_av * num
                count += num
            except (ValueError, TypeError, KeyError):
                pass
                    
if count == 0:
    average_rating = 0
else:
    average_rating = total_ratings / count

book_reviews = find_reviews_by_id(new_ids,)

# Combine into final record
book = {
    'id': new_ids,
    'title': story['title'],
    'author': story['author'],
    'story': story['text'],
    'av_rating': average_rating,
    'num_ratings':count,
    'reviews': book_reviews
}
#print(book)
data_3.append(book)

The History of the Necronomicon


#### The orator

In [690]:
still_unfound[98]['title']

'The Orator'

In [692]:
for item in BOOKS:
    if item['title'] == 'The Orator':
        names = []
        for aut_id in item['author_ids']:
            name = find_name_by_id(aut_id)
            if name:
                names.append(name)
        print(item['book_id'], names)

15806323 ['Edgar Wallace']


In [121]:
ac = find_books_by_author('Anton Chekhov',)
matches = [s for s in ac if 'orator' in s[0].lower()]
print(matches)

[]


#### The Past Was Goodly Once

In [699]:
still_unfound[99]['title']

'The Past Was Goodly Once'

In [703]:
matches = [s for s in weh if 'once' in s[0].lower()]
print(matches)

[]


#### The Wee Bannock

In [705]:
still_unfound[100]['title']

'The Wee Bannock'

In [706]:
with open("all_data_together_data3_100.jsonl", "w", encoding="utf-8") as f:
        for book in data_3:
            f.write(json.dumps(book, ensure_ascii=False) + "\n")

#### A Bush Christening

In [708]:
still_unfound[101]['title']

'A Bush Christening'

#### The pendulum

In [710]:
still_unfound[102]['title']

'The Pendulum'

In [122]:
for item in BOOKS:
    if item['title'] == 'The Pendulum':
        names = []
        for aut_id in item['author_ids']:
            name = find_name_by_id(aut_id)
            if name:
                names.append(name)
        print(item['book_id'], names)

11543875 ['Anne Elizabeth']


In [123]:
rb =find_books_by_author('Ray Bradbury',)
matches = [s for s in rb if 'pen' in s[0].lower()]
print(matches) 

[['The Penguin Book of Horror Stories', '1161149'], ['The Penguin Book of Ghost Stories', '697429'], ['Dark Forces: New Stories of Suspense and Supernatural Horror', '973196'], ['Pendulum', '15711977']]


In [124]:
new_ids = ['15711977']
story = still_unfound[102]
print(story['title'])
total_ratings = 0
count = 0
for id in new_ids:
    for item in BOOKS:
        if id == item['book_id']:
            try:
                rating_av = float(item['average_rating'])
                num = int(item['ratings_count'])
                total_ratings += rating_av * num
                count += num
            except (ValueError, TypeError, KeyError):
                pass
                    
if count == 0:
    average_rating = 0
else:
    average_rating = total_ratings / count


book_reviews = find_reviews_by_id(new_ids,)

# Combine into final record
book = {
    'id': new_ids,
    'title': story['title'],
    'author': story['author'],
    'story': story['text'],
    'av_rating': average_rating,
    'num_ratings':count,
    'reviews': book_reviews
}
#print(book)
data_3.append(book)

The Pendulum


#### The eagle the cat and the wild sow

In [717]:
still_unfound[103]['title']

'The Eagle, the Cat, and the Wild Sow'

In [718]:
matches = [s for s in ae if 'eagle' in s[0].lower()]
print(matches) 

[['The Tortoise And The Eagle', '36465817'], ['The Fighting Cocks And The Eagle', '32808763'], ['The Eagle And The Fox', '36434944']]


#### A dream lies dead

In [719]:
still_unfound[104]#['title']

{'title': 'A Dream Lies Dead',
 'author': 'Dorothy Parker',
 'text': 'A dream lies dead here. May you softly go\n\nBefore this place, and turn away your eyes,\n\nNor seek to know the look of that which dies\n\nImportuning Life for life. Walk not in woe,\n\nBut, for a little, let your step be slow.\n\nAnd, of your mercy, be not sweetly wise\n\nWith words of hope and Spring and tenderer skies.\n\nA dream lies dead; and this all mourners know:\n\nWhenever one drifted petal leaves the tree-\n\nThough white of bloom as it had been before\n\nAnd proudly waitful of fecundity-\n\nOne little loveliness can be no more;\n\nAnd so must Beauty bow her imperfect head\n\nBecause a dream has joined the wistful dead!',
 'word_count': 114,
 'url': 'https://www.libraryofshortstories.com/onlinereader/a-dream-lies-dead'}

In [126]:
dp = find_books_by_author('Dorothy Parker',)
matches = [s for s in dp if 'lies' in s[0].lower()]
print(matches) 

[['Here Lies: The Collected Stories of Dorothy Parker', '98856']]


#### The Man Who Was Away

In [724]:
still_unfound[105]['title']

'The Man Who Was Away'

#### The Detective Detector

In [726]:
still_unfound[106]['title']

'The Detective Detector'

In [730]:
oh = find_BOOKS_by_author('O. Henry',BOOKS, AUTHORS)
matches = [s for s in oh if 'det' in s[0].lower()]
print(matches) 

[['Detectives y ladrones', '30770027']]


#### Hunting the deceitful turkey

In [732]:
still_unfound[107]['title']

'Hunting the Deceitful Turkey'

In [735]:
matches = [s for s in mt if 'turkey' in s[0].lower()]
print(matches) 

[]


#### A death bed

In [737]:
still_unfound[108]['title']

'A Death Bed'

In [739]:
matches = [s for s in rk if 'bed' in s[0].lower()]
print(matches) 

[['The Butterfly That Stamped (Kipling, Rudyard, Just So Stories (Peter Bedrick Books).)', '490113']]


#### The Bear and the Travellers

In [741]:
still_unfound[109]['title']

'The Bear and the Travellers'

In [742]:
matches = [s for s in ae if 'bear' in s[0].lower()]
print(matches) 

[['The Bear And The Fox', '36437784']]


#### The Bat, the Bramble, and the Seagull

In [744]:
still_unfound[110]['title']

'The Bat, the Bramble, and the Seagull'

In [745]:
matches = [s for s in ae if 'bat' in s[0].lower()]
print(matches) 

[]


#### an incident

In [748]:
still_unfound[111]['title']

'An Incident'

In [750]:
matches = [s for s in ac if 'incident' in s[0].lower()]
print(matches) 

[]


#### Mesopotamia

In [753]:
still_unfound[112]['title']

'Mesopotamia'

In [755]:
matches = [s for s in rk if 'meso' in s[0].lower()]
print(matches) 

[]


#### the drover's sweetheart

In [757]:
still_unfound[113]['title']

'The Drover’s Sweetheart'

In [760]:
matches = [s for s in hl if 'sweet' in s[0].lower()]
print(matches) 

[]


#### the Daylight Is Dying

In [762]:
still_unfound[114]['title']

'The Daylight Is Dying'

#### Song of the Future

In [764]:
still_unfound[115]['title']

'Song of the Future'

#### a chameleon

In [767]:
still_unfound[116]['title']

'A Chameleon'

In [127]:
matches = [s for s in ac if 'cham' in s[0].lower()]
print(matches) 

[['Champagne', '18142691'], ['Chameleon', '16170403'], ['The Chameleon and Other Stories (with notes and biography) (illustrated)', '24993017']]


In [128]:
new_ids = ['16170403']
story = still_unfound[116]
print(story['title'])
total_ratings = 0
count = 0
for id in new_ids:
    for item in BOOKS:
        if id == item['book_id']:
            try:
                rating_av = float(item['average_rating'])
                num = int(item['ratings_count'])
                total_ratings += rating_av * num
                count += num
            except (ValueError, TypeError, KeyError):
                pass
                    
if count == 0:
    average_rating = 0
else:
    average_rating = total_ratings / count

book_reviews = find_reviews_by_id(new_ids,)

# Combine into final record
book = {
    'id': new_ids,
    'title': story['title'],
    'author': story['author'],
    'story': story['text'],
    'av_rating': average_rating,
    'num_ratings':count,
    'reviews': book_reviews
}
#print(book)
data_3.append(book)

A Chameleon


#### fhe Open Steeplechase

In [774]:
still_unfound[117]['title']

'The Open Steeplechase'

#### the old australian ways

In [776]:
still_unfound[118]['title']

'The Old Australian Ways'

In [876]:
matches = [s for s in abp if 'way' in s[0].lower()]
print(abp)

[['The Merino Sheep', '33391789'], ['Complete Poems', '745346'], ['Twas the Night Before Christmas and Other Christmas Stories', '8468620'], ["We're All Australians Now", '31847636'], ['Poems of Banjo Paterson', '20995192'], ["We're All Australians Now", '24358240'], ['The Cat', '23633435'], ['Waltzing Matilda', '111281'], ['The Best Of Banjo Patterson: An Illustrated Collection', '3172903'], ['The Man from Snowy River', '1841075'], ['Saltbush Bill, J. P', '12255399'], ['The Man From Snowy River', '6072732'], ["The Banjo's Best-Loved Poems", '11480077'], ["Works of 'Banjo' Paterson", '2746647'], ['The Man from Snowy River and Other Verses', '2725644'], ["Mulga Bill's Bicycle", '1651879'], ['The Man from Snowy River', '7902381'], ['An Outback Marriage', '6512349'], ['Clancy Of The Overflow', '397268'], ['Waltzing Matilda', '16074139'], ['The Merino Sheep', '33391789'], ['Complete Poems', '745346'], ['Twas the Night Before Christmas and Other Christmas Stories', '8468620'], ["We're All A

#### the poets of the tomb

In [781]:
still_unfound[119]['title']

'The Poets of the Tomb'

In [783]:
matches = [s for s in hl if 'poe' in s[0].lower()]
print(matches) 

[['Henry Lawson Illustrated Poems', '4279459'], ['Poetical Works', '8486481']]


#### The mystery of dave regan

In [786]:
still_unfound[120]['title']

'The Mystery of Dave Regan'

In [788]:
matches = [s for s in hl if 'dave' in s[0].lower()]
print(matches) 

[]


#### Tattercoats

In [790]:
still_unfound[121]['title']

'Tattercoats'

In [791]:
matches = [s for s in fas if 'tat' in s[0].lower()]
print(matches) 

[]


#### johnson's antidote

In [793]:
still_unfound[122]['title']

'Johnson’s Antidote'

#### been there before

In [795]:
still_unfound[123]['title']

'Been There Before'

#### clair de lune

In [797]:
still_unfound[124]['title']

'Clair De Lune'

In [129]:
matches = [s for s in gdm if 'lune' in s[0].lower()]
print(matches)

[['Clair de lune et autres nouvelles', '6011897'], ["Clair de lune : Clair de lune - Un coup d'Etat - Le loup - L'enfant - Conte de Noël - La reine Hortense - Le pardon - La légende du Mont Saint-Michel - ... - Nos lettres - La nuit", '19020055'], ['Clair de Lune', '27421730'], ['Clair de lune et autres nouvelles', '1667107']]


In [130]:
new_ids = ['27421730']
story = still_unfound[124]
print(story['title'])
total_ratings = 0
count = 0
for id in new_ids:
    for item in BOOKS:
        if id == item['book_id']:
            try:
                rating_av = float(item['average_rating'])
                num = int(item['ratings_count'])
                total_ratings += rating_av * num
                count += num
            except (ValueError, TypeError, KeyError):
                pass
                    
if count == 0:
    average_rating = 0
else:
    average_rating = total_ratings / count


book_reviews = find_reviews_by_id(new_ids,)

# Combine into final record
book = {
    'id': new_ids,
    'title': story['title'],
    'author': story['author'],
    'story': story['text'],
    'av_rating': average_rating,
    'num_ratings':count,
    'reviews': book_reviews
}
#print(book)
data_3.append(book)

Clair De Lune


#### roaring days

In [803]:
still_unfound[125]['title']

'The Roaring Days'

In [807]:
matches = [s for s in hl if 'roar' in s[0].lower()]
print(matches)

[]


#### The female of the species

In [809]:
still_unfound[126]['title']

'The Female of the Species'

In [812]:
matches = [s for s in rk if 'spec' in s[0].lower()]
print(matches)

[['Kim - Rudyard Kipling [Special edition] (Annotated)', '36149221']]


#### Happiness

In [814]:
still_unfound[127]['title']

'Happiness'

#### Only a jockey

In [816]:
still_unfound[128]['title']

'Only a Jockey'

#### the red ettin'

In [818]:
still_unfound[129]['title']

'The Red Ettin'

#### the geebung polo club

In [820]:
still_unfound[130]['title']

'The Geebung Polo Club'

#### henny penny

In [822]:
still_unfound[131]['title']

'Henny Penny'

In [830]:
matches = [s for s in fas if 'hen' in s[0].lower()]
print(matches)

[]


#### the belly and the members

In [825]:
still_unfound[132]['title']

'The Belly and the Members'

In [827]:
matches = [s for s in ae if 'belly' in s[0].lower()]
print(matches)

[]


#### the wise men of gotham

In [831]:
still_unfound[133]['title']

'The Wise Men Of Gotham'

#### the great calamity

In [833]:
still_unfound[134]['title']

'The Great Calamity'

#### oh the public

In [835]:
still_unfound[135]['title']

'Oh! The Public'

In [837]:
matches = [s for s in ac if 'public' in s[0].lower()]
print(matches)

[]


#### bill the ventriloquial rooster

In [839]:
still_unfound[136]['title']

'Bill, the Ventriloquial Rooster'

In [840]:
matches = [s for s in hl if 'bill' in s[0].lower()]
print(matches)

[['While the Billy Boils', '9282840'], ['While the Billy Boils', '18668111']]


#### the brogue

In [842]:
still_unfound[137]['title']

'The Brogue'

In [844]:
matches = [s for s in saki if 'brog' in s[0].lower()]
print(matches)

[]


#### a fairly sad tale

In [846]:
still_unfound[138]['title']

'A Fairly Sad Tale'

In [849]:
matches = [s for s in dp if 'sad' in s[0].lower()]
print(matches)

[]


#### a mountain station

In [851]:
still_unfound[139]['title']

'A Mountain Station'

#### the spring my dear

In [853]:
still_unfound[140]['title']

'The Spring My Dear'

In [855]:
matches = [s for s in weh if 'spring' in s[0].lower()]
print(matches)

[]


#### little red riding hood

In [857]:
still_unfound[141]['title']

'Little Red Riding-Hood'

#### lazy jack

In [860]:
still_unfound[142]['title']

'Lazy Jack'

#### the three sillies

In [863]:
still_unfound[143]['title']

'The Three Sillies'

#### our new horse

In [873]:
still_unfound[144]['title']
matches = [s for s in abp if 'horse' in s[0].lower()]
print(matches)

[]


#### the man from ironbark

In [867]:
still_unfound[145]['title']

'The Man from Ironbark'

In [872]:
abp.extend(find_BOOKS_by_author('A.B. Paterson', BOOKS,AUTHORS))
matches = [s for s in abp if 'man' in s[0].lower()]
print(matches)

[['The Man from Snowy River', '1841075'], ['The Man From Snowy River', '6072732'], ['The Man from Snowy River and Other Verses', '2725644'], ['The Man from Snowy River', '7902381'], ['The Man from Snowy River', '1841075'], ['The Man From Snowy River', '6072732'], ['The Man from Snowy River and Other Verses', '2725644'], ['The Man from Snowy River', '7902381']]


#### how the favourite beat us up

In [878]:
still_unfound[146]['title']

'How the Favourite Beat Us'

In [880]:
matches = [s for s in abp if 'how' in s[0].lower()]
print(matches)

[]


#### a certain lady

In [882]:
still_unfound[147]['title']

'A Certain Lady'

In [883]:
matches = [s for s in dp if 'lady' in s[0].lower()]
print(matches)

[]


#### how mginnis went missing

In [885]:
still_unfound[148]['title']

'How M’Ginnis Went Missing'

In [886]:
matches = [s for s in abp if 'went' in s[0].lower()]
print(matches)

[]


#### to the sun

In [888]:
still_unfound[149]['title']

'To the Sun'

In [131]:
ash = find_books_by_author('Clark Ashton Smith')
matches = [s for s in ash if 'sun' in s[0].lower()]
print(matches)

[]


#### the puppet show man

In [894]:
still_unfound[150]['title']

'The Puppet-show Man'

In [896]:
matches = [s for s in hca if 'puppet' in s[0].lower()]
print(matches)

[]


#### the fish and the ring

In [898]:
still_unfound[151]['title']

'The Fish and the Ring'

#### last week

In [900]:
still_unfound[152]['title']

'Last Week'

In [901]:
matches = [s for s in abp if 'last' in s[0].lower()]
print(matches)

[]


#### the ant

In [906]:
still_unfound[153]['title']

'The Ant'

In [904]:
matches = [s for s in ae if 'ant' in s[0].lower()]
print(matches)

[['The Ants And The Grasshopper', '31861954'], ["Walt Disney's: The Grasshopper and the Ants", '237374'], ['The Ant and the Grasshopper', '584042'], ['The Ant And The Grasshopper', '584049']]


#### the flying gang

In [908]:
still_unfound[154]['title']

'The Flying Gang'

In [909]:
matches = [s for s in abp if 'gang' in s[0].lower()]
print(matches)

[]


#### mrs packletides tiger

In [911]:
still_unfound[155]['title']

'Mrs. Packletide’s Tiger'

In [912]:
matches = [s for s in saki if 'tiger' in s[0].lower()]
print(matches)

[["Mrs. Packletide's Tiger", '1882973'], ["Mrs. Packletide's Tiger", '21642850']]


In [132]:
for item in BOOKS:
    if item['book_id'] == '21642850':
        names = []
        for aut_id in item['author_ids']:
            name =find_name_by_id(aut_id)
            names.append(name)
        print(names, item['book_id'])

['Saki'] 21642850


In [133]:
new_ids = ['21642850']
story = still_unfound[155]
print(story['title'])
total_ratings = 0
count = 0
for id in new_ids:
    for item in BOOKS:
        if id == item['book_id']:
            try:
                rating_av = float(item['average_rating'])
                num = int(item['ratings_count'])
                total_ratings += rating_av * num
                count += num
            except (ValueError, TypeError, KeyError):
                pass
                    
if count == 0:
    average_rating = 0
else:
    average_rating = total_ratings / count


book_reviews = find_reviews_by_id(new_ids,)

# Combine into final record
book = {
    'id': new_ids,
    'title': story['title'],
    'author': story['author'],
    'story': story['text'],
    'av_rating': average_rating,
    'num_ratings':count,
    'reviews': book_reviews
}
#print(book)
data_3.append(book)

Mrs. Packletide’s Tiger


#### the triumphs of a taxidermist

In [919]:
still_unfound[156]['title']

'The Triumphs of a Taxidermist'

In [920]:
for item in BOOKS:
    if item['title'] == 'The Triumphs of a Taxidermist':
        print(item['book_id'])

In [134]:
story = still_unfound[156]

# Combine into final record
book = {
    'id': new_ids,
    'title': story['title'],
    'author': story['author'],
    'story': story['text'],
    'av_rating': 3.05,
    'num_ratings':39,
    'reviews': [{'rating':4,'text':"The rich can get anything they want...hahaha! Pretty cool story."},
               {'rating':3,'text':"Very short. Clever, punchy, amusing... a good intro. for ppl. who aren't aware how good Wells could be at either short stories or humor."},
               {'rating':2,'text':"Most forgettable story by him so far."}]
}
#print(book)
data_3.append(book)

#### the lion and the mouse

In [924]:
still_unfound[157]['title']

'The Lion and the Mouse'

In [927]:
matches = [s for s in ae if 'mouse' in s[0].lower()]
print(matches)

[['The Country Mouse And The Town Mouse', '36454719'], ['Town Mouse & Country Mouse', '1827075'], ['The Town Mouse and the Country Mouse: The Boy Who Cried Wolf', '1275099'], ['The Town Mouse and the Country Mouse', '5449950']]


#### conroy's gap

In [929]:
still_unfound[158]['title']

'Conroy’s Gap'

In [930]:
matches = [s for s in abp if 'gap' in s[0].lower()]
print(matches)

[]


#### the eagle and the cocks

In [932]:
still_unfound[159]['title']

'The Eagle and the Cocks'

In [933]:
matches = [s for s in ae if 'eagle' in s[0].lower()]
print(matches)

[['The Tortoise And The Eagle', '36465817'], ['The Fighting Cocks And The Eagle', '32808763'], ['The Eagle And The Fox', '36434944']]


In [135]:
new_ids = ['32808763']
story = still_unfound[159]
print(story['title'])
total_ratings = 0
count = 0
for id in new_ids:
    for item in BOOKS:
        if id == item['book_id']:
            try:
                rating_av = float(item['average_rating'])
                num = int(item['ratings_count'])
                total_ratings += rating_av * num
                count += num
            except (ValueError, TypeError, KeyError):
                pass
                    
if count == 0:
    average_rating = 0
else:
    average_rating = total_ratings / count


book_reviews = find_reviews_by_id(new_ids,)

# Combine into final record
book = {
    'id': new_ids,
    'title': story['title'],
    'author': story['author'],
    'story': story['text'],
    'av_rating': average_rating,
    'num_ratings':count,
    'reviews': book_reviews
}
#print(book)
data_3.append(book)

The Eagle and the Cocks


#### the boston massacre

In [937]:
still_unfound[160]['title']

'The Boston Massacre'

In [940]:
nh=find_BOOKS_by_author('Nathaniel Hawthorne',BOOKS,AUTHORS)
matches = [s for s in nh if 'boston' in s[0].lower()]
print(matches)

[]


#### the story of the three bears

In [942]:
still_unfound[161]['title']

'The Story Of The Three Bears'

In [943]:
matches = [s for s in fas if 'bears' in s[0].lower()]
print(matches)

[['Goldilocks and the Three Bears', '20556960']]


In [136]:
new_ids = ['20556960']
story = still_unfound[161]
print(story['title'])
total_ratings = 0
count = 0
for id in new_ids:
    for item in BOOKS:
        if id == item['book_id']:
            try:
                rating_av = float(item['average_rating'])
                num = int(item['ratings_count'])
                total_ratings += rating_av * num
                count += num
            except (ValueError, TypeError, KeyError):
                pass
                    
if count == 0:
    average_rating = 0
else:
    average_rating = total_ratings / count


book_reviews = find_reviews_by_id(new_ids,)

# Combine into final record
book = {
    'id': new_ids,
    'title': story['title'],
    'author': story['author'],
    'story': story['text'],
    'av_rating': average_rating,
    'num_ratings':count,
    'reviews': book_reviews
}
#print(book)
data_3.append(book)

The Story Of The Three Bears


#### in answer to various bards

In [947]:
still_unfound[162]['title']

'In Answer to Various Bards'

In [948]:
matches = [s for s in abp if 'bards' in s[0].lower()]
print(matches)

[]


#### shearing at castlereagh

In [950]:
still_unfound[163]['title']

'Shearing at Castlereagh'

In [951]:
matches = [s for s in abp if 'shear' in s[0].lower()]
print(matches)

[]


#### the all right un

In [954]:
still_unfound[164]['title']

'The All Right ’Un'

In [955]:
matches = [s for s in abp if 'all' in s[0].lower()]
print(matches)

[["We're All Australians Now", '31847636'], ["We're All Australians Now", '24358240'], ["We're All Australians Now", '31847636'], ["We're All Australians Now", '24358240']]


#### a deal in ostriches

In [957]:
still_unfound[165]['title']

'A Deal In Ostriches'

In [962]:
for item in BOOKS:
    if item['title'] == 'A Deal in Ostriches':
        print(item['book_id'])

32877355


In [961]:
matches = [s for s in hgw if 'deal' in s[0].lower()]
print(matches)

[['A Deal in Ostriches', '32877355']]


In [137]:
new_ids = ['32877355']
story = still_unfound[165]
print(story['title'])
total_ratings = 0
count = 0
for id in new_ids:
    for item in BOOKS:
        if id == item['book_id']:
            try:
                rating_av = float(item['average_rating'])
                num = int(item['ratings_count'])
                total_ratings += rating_av * num
                count += num
            except (ValueError, TypeError, KeyError):
                pass
                    
if count == 0:
    average_rating = 0
else:
    average_rating = total_ratings / count


book_reviews = find_reviews_by_id(new_ids,)

# Combine into final record
book = {
    'id': new_ids,
    'title': story['title'],
    'author': story['author'],
    'story': story['text'],
    'av_rating': average_rating,
    'num_ratings':count,
    'reviews': book_reviews
}
#print(book)
data_3.append(book)

A Deal In Ostriches


#### the hen

In [966]:
still_unfound[166]['title']

'The Hen'

In [967]:
matches = [s for s in saki if 'hen' in s[0].lower()]
print(matches)

[['When William Came', '525598']]


#### the eagle and the beetle

In [969]:
still_unfound[167]['title']

'The Eagle and the Beetle'

In [970]:
matches = [s for s in ae if 'eagle' in s[0].lower()]
print(matches)

[['The Tortoise And The Eagle', '36465817'], ['The Fighting Cocks And The Eagle', '32808763'], ['The Eagle And The Fox', '36434944']]


#### a bunch of roses

In [972]:
still_unfound[168]['title']

'A Bunch of Roses'

In [973]:
matches = [s for s in abp if 'rose' in s[0].lower()]
print(matches)

[]


#### a bushman's song

In [975]:
still_unfound[169]['title']

'A Bushman’s Song'

In [976]:
matches = [s for s in abp if 'song' in s[0].lower()]
print(matches)

[]


#### an extinct angel

In [978]:
still_unfound[170]['title']

'An Extinct Angel'

In [980]:
for item in BOOKS:
    if item['title'] == 'An extinct Angel':
        print(item['book_id'])

In [982]:
cpg=find_BOOKS_by_author('Charlotte Perkins Gilman',BOOKS,AUTHORS)

In [139]:
story = still_unfound[170]
# Combine into final record
book = {
    'id': new_ids,
    'title': story['title'],
    'author': story['author'],
    'story': story['text'],
    'av_rating': 3.65,
    'num_ratings': 74,
    'reviews': [{'rating':3,'text':"Pulled the expectations of the patriarchy into sight, but lacked subtlety and insight. It seems awful to be nitpicking at the writing, but while the subject and jibes were spot on, but the reading experience did not leave me with any new perspective."},
               {'rating':3,'text':"3.25 An Extinct Angel is definitely a feminist writing, not really a short story at all. It has an undertone of humorous bitterness that I enjoyed. However it didn't really strike me as incredibly powerful or moving (coming from someone who really enjoys feminist material). Taking into consideration the time it was written, this was likely very unique and daring for the time, but in more modern times, it's now become a little tame. It doesn't dive very deep nor was there much I felt a deep connection to. There's nothing wrong with it at all, but it probably won't stick with me. It was just okay!"},
               {'rating':4,'text':"'he never assisted the angel to rise, but got out from under and resumed his way, leaving her in the mud. she was a great convenience to walk on, and, as was stoutly maintained by the human creature, helped to keep the other angels clean' i teared up reading that part"},]
}
#print(book)
data_3.append(book)

#### the ass and the mule

In [987]:
still_unfound[171]['title']

'The Ass and the Mule'

In [989]:
matches = [s for s in ae if 'mule' in s[0].lower()]
print(matches)

[['The Mule', '33141885']]


#### the well of the world's end

In [991]:
still_unfound[172]['title']

'The Well of the World’s End'

#### over the range

In [993]:
still_unfound[173]['title']

'Over the Range'

In [994]:
matches = [s for s in abp if 'range' in s[0].lower()]
print(matches)

[]


#### Allez

In [996]:
still_unfound[174]['title']

'“allez!”'

In [1000]:
aik.extend(find_BOOKS_by_author('Aleksandr Kuprin',BOOKS,AUTHORS))
matches = [s for s in aik if 'al' in s[0].lower()]
print(aik)

[]


In [141]:
story = still_unfound[174]
# Combine into final record
book = {
    'id': [],
    'title': 'Allez!',
    'author': story['author'],
    'story': story['text'],
    'av_rating': 4.33,
    'reviews': [{'rating':4,'text':"Allez!"}]
}
#print(book)
data_3.append(book)

#### in the droving days

In [1007]:
still_unfound[175]['title']

'In the Droving Days'

In [1008]:
matches = [s for s in abp if 'day' in s[0].lower()]
print(matches)

[]


#### A tent in agony

In [1010]:
still_unfound[176]['title']

'A Tent in Agony'

#### the garden of the holy virgin

In [1012]:
still_unfound[177]['title']

'The Garden Of The Holy Virgin'

#### the golden ball

In [1014]:
still_unfound[178]['title']

'The Golden Ball'

In [1015]:
matches = [s for s in fas if 'ball' in s[0].lower()]
print(matches)

[]


#### 'Sez You

In [1018]:
still_unfound[179]['title']

'‘Sez You’'

In [1020]:
matches = [s for s in hl if 'you' in s[0].lower()]
print(matches)

[]


#### the fox and the grapes

In [1022]:
still_unfound[180]['title']

'The Fox and the Grapes'

In [1023]:
matches = [s for s in ae if 'grape' in s[0].lower()]
print(matches)

[['The Fox And The Grapes', '33131659']]


In [142]:
for item in BOOKS:
    names = []
    if item['book_id'] == '33131659':
        for id in item['author_ids']:
            name = find_name_by_id(id,)
            names.append(name)
        print(names)

['Aesop']


In [143]:
new_ids = ['33131659']
story = still_unfound[180]
print(story['title'])
total_ratings = 0
count = 0
for id in new_ids:
    for item in BOOKS:
        if id == item['book_id']:
            try:
                rating_av = float(item['average_rating'])
                num = int(item['ratings_count'])
                total_ratings += rating_av * num
                count += num
            except (ValueError, TypeError, KeyError):
                pass
                    
if count == 0:
    average_rating = 0
else:
    average_rating = total_ratings / count


book_reviews = find_reviews_by_id(new_ids,)

# Combine into final record
book = {
    'id': new_ids,
    'title': story['title'],
    'author': story['author'],
    'story': story['text'],
    'av_rating': average_rating,
    'num_ratings':count,
    'reviews': book_reviews
}
#print(book)
data_3.append(book)

The Fox and the Grapes


#### the boss of the admiral lynch

In [1029]:
still_unfound[181]['title']

'The Boss of the ‘Admiral Lynch’'

In [1030]:
matches = [s for s in abp if 'boss' in s[0].lower()]
print(matches)

[]


#### the city bushman

In [1032]:
still_unfound[182]['title']

'The City Bushman'

In [1033]:
matches = [s for s in hl if 'bush' in s[0].lower()]
print(matches)

[]


#### a defenceless creature

In [1035]:
still_unfound[183]['title']

'A Defenceless Creature'

In [1038]:
matches = [s for s in ac if 'creature' in s[0].lower()]
print(matches)

[]


#### a charm

In [1042]:
still_unfound[184]['title']

'A Charm'

In [1044]:
matches = [s for s in rk if 'charm' in s[0].lower()]
print(matches)

[]


#### 39

In [1046]:
still_unfound[185]['title']

'39'

In [1047]:
matches = [s for s in hl if '39' in s[0].lower()]
print(matches)

[]


#### the swagman's rest

In [1049]:
still_unfound[186]['title']

'The Swagman’s Rest'

In [1050]:
matches = [s for s in abp if 'rest' in s[0].lower()]
print(matches)

[]


#### the temptation of harringay

In [1052]:
still_unfound[187]['title']

'The Temptation of Harringay'

In [1056]:
matches = [s for s in hgw if 'tempt' in s[0].lower()]
print(matches)

[]


In [1057]:
for item in BOOKS:
    if item['title'] == 'The Temptation of Harringay':
        print(item['book_id'])

#### the travelling post office

In [1061]:
still_unfound[188]['title']

'The Travelling Post Office'

In [1062]:
matches = [s for s in abp if 'post' in s[0].lower()]
print(matches)

[]


#### babes in the wood

In [1064]:
still_unfound[189]['title']

'The Babes In The Wood'

#### mr fox

In [1066]:
still_unfound[190]['title']

'Mr. Fox'

In [1069]:
matches = [s for s in fas if 'fox' in s[0].lower()]
print(matches)

[]


#### The song and the dance

In [1073]:
still_unfound[191]['title']

'The Song And The Dance'

#### the three little pigs

In [1075]:
still_unfound[192]['title']

'The Three Little Pigs'

#### the amateur gardener

In [1077]:
still_unfound[193]['title']

'The Amateur Gardener'

In [1078]:
matches = [s for s in abp if 'gard' in s[0].lower()]
print(matches)

[]


#### borderland

In [1080]:
still_unfound[194]['title']

'Borderland'

In [1081]:
matches = [s for s in hl if 'land' in s[0].lower()]
print(matches)

[]


In [ ]:
still_unfound[195]['title']

#### save data

In [144]:
len(data_3)

124

In [153]:
def has_repeated_dict(list_of_dicts):
    seen_dicts = set()
    for d in list_of_dicts:
        title = d['title']
        if title in seen_dicts:
            return True, title
        seen_dicts.add(title)
    return False

In [152]:
has_repeated_dict(data_3)

(True, 'Hans Christian Andersen')

In [154]:
with open("all_data_together_with_counts.jsonl", "w", encoding="utf-8") as f:
        for book in data_3:
            f.write(json.dumps(book, ensure_ascii=False) + "\n")